In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import sklearn
from sklearn import model_selection as ms
from sklearn.linear_model import LogisticRegression

# sns.set(rc={'figure.figsize':(11.7,8.27)})

import gzip

import copy

import import_ipynb
from My_Functions import null_cols, rmse

importing Jupyter notebook from My_Functions.ipynb


In [2]:
acc = pd.read_csv('../../accepted_2007_to_2018Q4.csv.gz',
                  compression='gzip',
                  header=0,
                  sep=',',
                  quotechar='"')
acc

C:\Users\tdcho\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

In [3]:
# we will remove those rows
rows_to_drop_id = list(acc[acc['id'].str.contains('Total|Loans', na=False)].index)

acc = acc.drop(rows_to_drop_id)
acc

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.00,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.00,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN

In [4]:
# getting 70% of principal
acc[['principal_70']] = acc[['funded_amnt']] * .7

In [5]:
# dropping letters and converting strings to numeric
acc.loc[acc['term'].str.contains('36', na=False), 'term'] = 36
acc.loc[acc['term'].str.contains('60', na=False), 'term'] = 60

# converting the 'term' column to integer
acc['term'] = acc['term'].astype(int)

acc[['term']]

,term
0,36
1,36
2,60
3,60
4,60
...,...
2260694,60
2260695,60
2260696,60
2260697,60


In [6]:
# getting expected total amount from loan
acc['total_expected'] = acc['installment'] * acc['term']

# round to 2 decimal places
acc[['total_expected']] = acc[['total_expected']].round(2)

acc[['total_expected']]

,total_expected
0,4429.08
1,29530.08
2,25959.60
3,49794.00
4,17394.60
...,...
2260694,32610.00
2260695,30944.40
2260696,51573.60
2260697,33873.60


In [7]:
# getting 70% of total expected payments
acc[['total_expected_70']] = acc[['total_expected']] * .7

# round to 2 decimal places
acc[['total_expected_70']] = acc[['total_expected_70']].round(2)

acc[['total_expected_70']]

,total_expected_70
0,3100.36
1,20671.06
2,18171.72
3,34855.80
4,12176.22
...,...
2260694,22827.00
2260695,21661.08
2260696,36101.52
2260697,23711.52


In [8]:
acc[['funded_amnt', 'total_pymnt']]

,funded_amnt,total_pymnt
0,3600.0,4421.723917
1,24700.0,25679.660000
2,20000.0,22705.924294
3,35000.0,31464.010000
4,10400.0,11740.500000
...,...,...
2260694,24000.0,15760.500000
2260695,24000.0,15008.530000
2260696,40000.0,24903.930000
2260697,24000.0,6755.400000


### Principal 70%

In [9]:
# dataframe of total payment being less than 70% of principal
acc[acc['total_pymnt']<acc['principal_70']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,principal_70,total_expected,total_expected_70
13,66624733,NaN,18000.0,18000.0,18000.0,60,19.48,471.70,E,E2,Software Manager,7 years,RENT,150000.0,Not Verified,Dec-2015,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,916xx,CA,9.39,0.0,Jun-2005,665.0,669.0,1.0,NaN,75.0,18.0,1.0,14052.0,40.7,27.0,w,0.00,0.00,9452.74,9452.74,3481.86,4351.98,0.00,1618.90,291.4020,May-2017,471.70,NaN,Nov-2017,584.0,580.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,60.0,36247.0,2.0,2.0,1.0,3.0,2.0,22195.0,71.0,2.0,4.0,1271.0,55.0,34500.0,2.0,1.0,2.0,7.0,2014.0,5128.0,51.3,0.0,0.0,115.0,112.0,1.0,1.0,2.0,9.0,NaN,2.0,NaN,0.0,6.0,13.0,7.0,8.0,6.0,16.0,19.0,13.0,18.0,0.0,0.0,0.0,3.0,100.0,14.3,1.0,0.0,65819.0,36247.0,10300.0,31319.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,12600.0,28302.00,19811.40
25,67849662,NaN,4225.0,4225.0,4225.0,36,14.85,146.16,C,C5,mechanic,5 years,RENT,35000.0,Source Verified,Dec-2015,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,672xx,KS,15.22,2.0,Jul-2011,725.0,729.0,0.0,18.0,NaN,6.0,0.0,1058.0,24.6,6.0,w,0.00,0.00,2558.87,2558.87,1536.90,653.60,0.00,368.37,66.3066,Apr-2017,146.16,NaN,Dec-2017,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,4888.0,0.0,1.0,0.0,0.0,47.0,3830.0,22.0,0.0,0.0,367.0,22.0,4300.0,0.0,0.0,0.0,0.0,815.0,2233.0,14.1,0.0,0.0,47.0,53.0,26.0,26.0,0.0,26.0,NaN,14.0,18.0,0.0,1.0,2.0,3.0,3.0,1.0,5.0,5.0,2.0,6.0,0.0,0.0,0.

In [10]:
# dataframe of total payment being greater than 70% of principal
acc[acc['total_pymnt']>=acc['principal_70']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2520.0,4429.08,3100.36
1,68355089,NaN,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0

### Total payment 70%

In [11]:
# dataframe of total payment being less than 70% of total payment
acc[acc['total_pymnt']<acc['total_expected_70']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,principal_70,total_expected,total_expected_70
3,66310712,NaN,35000.0,35000.0,35000.0,60,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,076xx,NJ,17.06,0.0,Sep-2008,785.0,789.0,0.0,NaN,NaN,13.0,0.0,7802.0,11.6,17.0,w,15897.65,15897.65,31464.01,31464.01,19102.35,12361.66,0.00,0.0,0.000,Feb-2019,829.90,Apr-2019,Mar-2019,679.0,675.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,301500.0,1.0,1.0,0.0,1.0,23.0,12609.0,70.0,1.0,1.0,6987.0,45.0,67300.0,0.0,1.0,0.0,2.0,23192.0,54962.0,12.1,0.0,0.0,36.0,87.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,0.0,4.0,5.0,8.0,10.0,2.0,10.0,13.0,5.0,13.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,24500.0,49794.0,34855.80
4,68476807,NaN,10400.0,10400.0,10400.0,60,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,major_purchase,Major purchase,174xx,PA,25.37,1.0,Jun-1998,695.0,699.0,3.0,12.0,NaN,12.0,0.0,21929.0,64.5,35.0,w,0.00,0.00,11740.50,11740.50,10400.00,1340.50,0.00,0.0,0.000,Jul-2016,10128.96,NaN,Mar-2018,704.0,700.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,331730.0,1.0,3.0,0.0,3.0,14.0,73839.0,84.0,4.0,7.0,9702.0,78.0,34000.0,2.0,1.0,3.0,10.0,27644.0,4567.0,77.5,0.0,0.0,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,0.0,4

In [12]:
# dataframe of total payment being greater than 70% of total payment
acc[acc['total_pymnt']>=acc['total_expected_70']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2520.0,4429.08,3100.36
1,68355089,NaN,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,31401

## We can also look at if the investor can at least make back principal

In [13]:
# dataframe of total payment being less than principal
acc[acc['total_pymnt']<acc['funded_amnt']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,principal_70,total_expected,total_expected_70
3,66310712,NaN,35000.0,35000.0,35000.0,60,14.85,829.90,C,C5,Information Systems Officer,10+ years,MORTGAGE,110000.0,Source Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,076xx,NJ,17.06,0.0,Sep-2008,785.0,789.0,0.0,NaN,NaN,13.0,0.0,7802.0,11.6,17.0,w,15897.65,15897.65,31464.01,31464.01,19102.35,12361.66,0.00,0.0,0.000,Feb-2019,829.90,Apr-2019,Mar-2019,679.0,675.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,301500.0,1.0,1.0,0.0,1.0,23.0,12609.0,70.0,1.0,1.0,6987.0,45.0,67300.0,0.0,1.0,0.0,2.0,23192.0,54962.0,12.1,0.0,0.0,36.0,87.0,2.0,2.0,1.0,2.0,NaN,NaN,NaN,0.0,4.0,5.0,8.0,10.0,2.0,10.0,13.0,5.0,13.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,381215.0,52226.0,62500.0,18000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,24500.0,49794.0,34855.80
10,68356421,NaN,22400.0,22400.0,22400.0,60,12.88,508.30,C,C2,Executive Director,6 years,MORTGAGE,95000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,290xx,SC,22.98,0.0,Apr-1995,710.0,714.0,0.0,54.0,NaN,16.0,0.0,13028.0,32.7,35.0,w,9912.88,9912.88,19275.33,19275.33,12487.12,6788.21,0.00,0.0,0.000,Feb-2019,508.30,Apr-2019,Mar-2019,589.0,585.0,0.0,54.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,376218.0,1.0,5.0,0.0,3.0,17.0,171328.0,67.0,3.0,6.0,8121.0,54.0,39800.0,1.0,5.0,0.0,9.0,23514.0,12617.0,39.9,0.0,0.0,150.0,248.0,1.0,1.0,5.0,1.0,NaN,14.

In [14]:
# dataframe of total payment being greater than principal
acc[acc['total_pymnt']>=acc['funded_amnt']]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.0,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2520.0,4429.08,3100.36
1,68355089,NaN,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.0,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,31401

- we see that comparing total payment to funded amount (principal) gives us a more even split

In [15]:
# converting to 1s and 0s

# 1 means UN-able to recoup principal
acc[['investor_expect']] = 1

# 0 means ABLE to recoup principal
acc.loc[acc['total_pymnt']>=acc['funded_amnt'], 'investor_expect'] = 0

acc[['investor_expect']]

,investor_expect
0,0
1,0
2,0
3,1
4,0
...,...
2260694,1
2260695,1
2260696,1
2260697,1


# Data Cleaning and Imputation

## Getting rows that contain the desired target

In [16]:
# keeping only the rows that have loan status of Paid Off, Default, or Charged Off

acc_train = copy.deepcopy(acc[(acc['loan_status'].str.contains('Paid|Default|Charged', na=False))])
acc_train = acc_train.reset_index()
acc_train = acc_train.drop('index', axis=1)
acc_train

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,principal_70,total_expected,total_expected_70,investor_expect
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2520.0,4429.08,3100.36,0
1,68355089,NaN,24700.0,24700.0,24700.0,36,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.0,0.0,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97

In [17]:
# create target label

target_train = acc_train[['investor_expect']]

target_train

,investor_expect
0,0
1,0
2,0
3,0
4,0
...,...
1348094,0
1348095,0
1348096,1
1348097,0


In [18]:
target_train[['investor_expect']].value_counts()

investor_expect
0                  1097706
1                   250393
dtype: int64

In [19]:
acc_train = acc_train.drop('investor_expect', axis=1)

## Splitting the dataset into train and prediction

In [20]:
# getting indices of half of the samples

target_nodwnsmpl_index_0 = sorted(list(target_train\
                                       [target_train\
                                        ['investor_expect']==0].sample(frac=0.5,
                                                                          random_state=0).index))

target_nodwnsmpl_index_1 = sorted(list(target_train\
                                       [target_train\
                                        ['investor_expect']==1].sample(frac=0.5,
                                                                          random_state=0).index))

target_nodwnsmpl_index_0and1 = target_nodwnsmpl_index_0 + target_nodwnsmpl_index_1
target_nodwnsmpl_index_0and1

[0,
 5,
 6,
 7,
 8,
 14,
 15,
 19,
 20,
 22,
 25,
 26,
 31,
 34,
 35,
 37,
 41,
 42,
 51,
 53,
 56,
 60,
 61,
 64,
 66,
 69,
 70,
 71,
 72,
 73,
 74,
 76,
 81,
 84,
 85,
 89,
 93,
 95,
 96,
 97,
 98,
 101,
 103,
 104,
 108,
 109,
 112,
 114,
 115,
 118,
 120,
 124,
 126,
 131,
 133,
 140,
 142,
 150,
 152,
 153,
 158,
 162,
 164,
 168,
 169,
 170,
 172,
 173,
 175,
 176,
 180,
 181,
 182,
 183,
 185,
 186,
 190,
 192,
 195,
 196,
 199,
 200,
 201,
 204,
 206,
 208,
 209,
 215,
 217,
 219,
 222,
 223,
 225,
 227,
 228,
 232,
 235,
 238,
 239,
 246,
 248,
 255,
 257,
 258,
 262,
 263,
 264,
 267,
 269,
 275,
 276,
 278,
 279,
 282,
 291,
 295,
 296,
 297,
 298,
 301,
 306,
 307,
 308,
 309,
 318,
 322,
 324,
 326,
 327,
 328,
 339,
 340,
 342,
 343,
 344,
 349,
 352,
 354,
 355,
 356,
 357,
 358,
 359,
 362,
 363,
 364,
 366,
 369,
 375,
 376,
 380,
 381,
 382,
 383,
 385,
 386,
 389,
 390,
 395,
 397,
 399,
 402,
 407,
 409,
 412,
 416,
 417,
 418,
 422,
 423,
 424,
 426,
 427,
 429,
 4

In [21]:
len(target_nodwnsmpl_index_0and1)

674049

In [22]:
# getting the other half of the indices

target_nodwnsmpl_pred_index = sorted(list(set(list(range(0,
                                               target_train['investor_expect'].count()))) -\
                                set(target_nodwnsmpl_index_0and1)))

target_nodwnsmpl_pred_index

[1,
 2,
 3,
 4,
 9,
 11,
 12,
 13,
 16,
 17,
 18,
 21,
 23,
 24,
 27,
 28,
 29,
 30,
 32,
 33,
 36,
 38,
 39,
 40,
 43,
 44,
 45,
 46,
 47,
 48,
 50,
 54,
 57,
 58,
 59,
 62,
 63,
 65,
 67,
 75,
 77,
 78,
 79,
 80,
 82,
 87,
 88,
 90,
 91,
 92,
 94,
 99,
 102,
 105,
 106,
 110,
 113,
 116,
 117,
 119,
 123,
 125,
 127,
 128,
 129,
 130,
 132,
 134,
 135,
 137,
 139,
 141,
 143,
 144,
 145,
 146,
 147,
 149,
 151,
 154,
 155,
 156,
 157,
 159,
 161,
 163,
 165,
 166,
 167,
 171,
 174,
 177,
 179,
 184,
 187,
 189,
 191,
 193,
 194,
 198,
 202,
 203,
 205,
 207,
 210,
 212,
 214,
 216,
 218,
 220,
 221,
 224,
 226,
 229,
 230,
 231,
 233,
 234,
 236,
 237,
 240,
 242,
 243,
 244,
 245,
 247,
 249,
 250,
 251,
 252,
 253,
 254,
 256,
 260,
 261,
 265,
 266,
 268,
 270,
 271,
 272,
 273,
 274,
 277,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 292,
 293,
 299,
 302,
 303,
 304,
 310,
 311,
 312,
 314,
 315,
 316,
 317,
 319,
 320,
 321,
 323,
 325,
 329,
 330,
 331,
 332,
 333,
 334,
 

In [23]:
len(target_nodwnsmpl_pred_index)

674050

In [24]:
# separating the dataframe into 2 dataframes: 1 for use/train and 1 for prediction comparison

acc_3_train_downsample = acc_train.iloc[target_nodwnsmpl_index_0and1]
acc_3_train_downsample = acc_3_train_downsample.reset_index()
acc_3_train_downsample = acc_3_train_downsample.drop('index', axis=1)

target_3_train_downsample = target_train.iloc[target_nodwnsmpl_index_0and1]
target_3_train_downsample = target_3_train_downsample.reset_index()
target_3_train_downsample = target_3_train_downsample.drop('index', axis=1)


acc_3_pred_downsample = acc_train.iloc[target_nodwnsmpl_pred_index]
acc_3_pred_downsample = acc_3_pred_downsample.reset_index()
acc_3_pred_downsample = acc_3_pred_downsample.drop('index', axis=1)

target_3_pred_downsample = target_train.iloc[target_nodwnsmpl_pred_index]
target_3_pred_downsample = target_3_pred_downsample.reset_index()
target_3_pred_downsample = target_3_pred_downsample.drop('index', axis=1)


acc_3_train_downsample

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,principal_70,total_expected,total_expected_70
0,68407277,NaN,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.0,0.0,4421.723917,4421.72,3600.00,821.72,0.0,0.00,0.0000,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2520.0,4429.08,3100.36
1,68476668,NaN,20000.0,20000.0,20000.0,36,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,550xx,MN,14.67,0.0,Jun-1990,680.0,684.0,0.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,0.0,0.0,21393.800000,21393.80,20000.00,1393.80,0.0,0.00,0.0000,Nov-2016,15681.05,NaN,Mar-2019,654.0,650.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,0.0,2.0,0.0,2.0,18.0,29433.0,63.0,2.0,3.0,13048.0,74.0,94200.0,1.0,0.0,1.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,0.0,4.0,6.0,4.0,5.0,7.

In [25]:
target_3_train_downsample

,investor_expect
0,0
1,0
2,0
3,0
4,0
...,...
674044,1
674045,1
674046,1
674047,1


# SHOULD WE BE DROPPING ROWS RN?

In [26]:
# drop unneeded columns

acc_clean = acc_3_train_downsample.drop(['collection_recovery_fee', 'desc', 'id', 'member_id',
                                          'next_pymnt_d', 'out_prncp', 'out_prncp_inv','pymnt_plan',
                                          'recoveries', 'title', 'total_pymnt_inv',
                                          'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp',
                                          'url', 'hardship_flag', 'hardship_type', 'hardship_reason',
                                          'hardship_status', 'deferral_term', 'hardship_amount',
                                          'hardship_start_date', 'hardship_end_date',
                                          'payment_plan_start_date', 'hardship_length',
                                          'hardship_dpd', 'hardship_loan_status',
                                          'orig_projected_additional_accrued_interest',
                                          'hardship_payoff_balance_amount',
                                          'hardship_last_payment_amount', 'disbursement_method',
                                          'debt_settlement_flag', 'debt_settlement_flag_date',
                                          'settlement_status', 'settlement_date', 'settlement_amount',
                                          'settlement_percentage', 'settlement_term', 'last_pymnt_d',
                                          'last_pymnt_amnt', 'last_fico_range_high',
                                          'last_fico_range_low', 'inq_last_12m', 'inq_last_6mths',
                                          'open_rv_24m', 'open_rv_12m', 'open_il_24m', 'open_il_12m',
                                          'open_acc_6m', 'num_tl_op_past_12m', 'num_tl_90g_dpd_24m',
                                          'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                                          'sec_app_chargeoff_within_12_mths',
                                          'sec_app_collections_12_mths_ex_med', 'zip_code',
                                          'addr_state'], axis=1)

acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70
0,3600.0,3600.0,3600.0,36,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,5.91,0.0,Aug-2003,675.0,679.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,Mar-2019,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2520.0,4429.08,3100.36
1,20000.0,20000.0,20000.0,36,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,MORTGAGE,180000.0,Not Verified,Dec-2015,Fully Paid,debt_consolidation,14.67,0.0,Jun-1990,680.0,684.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14000.0,22952.88,16067.02
2,20000.0,20000.0,20000.0,36,8.49,631.26,B,B1,road driver,10+ years,MORTGAGE,85000.0,Not Verified,Dec-2015,Fully Paid,major_purchase,17.61,1.0,Feb-1999,705.0,709.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,Mar-2019,0.0,3.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14000.0,22725.36,15907.75
3,10000.0,10000.0,10000.0,36,6.49,306.45,A,A2,SERVICE MANAGER,6 years,RENT,85000.0,Not Verified,Dec-2015,Fully Paid,credit_card,13.07,0.0,Apr-2002,685.0,689.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,Mar-2019,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7000.0,11032.20,7722.54
4,8000.0,8000.0,8000.0,36,11.48,263.74,B,B5,Vendor liaison,10+ years,MORTGAGE,42000.0,Not Verified,Dec-2015,Fully Paid,credit_card,34.80,0.0,Nov-1994,700.0,704.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,Nov-2018,0.0,75.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0

## Employment Length

In [27]:
acc_clean['emp_length'].unique()

array(['10+ years', '6 years', '8 years', '2 years', '9 years',
       '< 1 year', '5 years', nan, '1 year', '4 years', '3 years',
       '7 years'], dtype=object)

In [28]:
# if employment length is 10+ years, insert 10 years

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='10+ years',
                                   '10 years', acc_clean['emp_length'])


# if employment length is 1 year , insert 1 years

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='1 year',
                                   '1 years', acc_clean['emp_length'])


# if employment length is <1, insert 0

acc_clean['emp_length'] = np.where(acc_clean['emp_length']=='< 1 year',
                                   '0 years', acc_clean['emp_length'])


# if employment length is null, insert -1

acc_clean['emp_length'] = np.where(acc_clean['emp_length'].isnull(),
                                   '-1 years', acc_clean['emp_length'])

acc_clean['emp_length']

0         10 years
1         10 years
2         10 years
3          6 years
4         10 years
            ...   
674044     1 years
674045    10 years
674046     2 years
674047    10 years
674048     6 years
Name: emp_length, Length: 674049, dtype: object

In [29]:
# removing ' years' from the column

acc_clean['emp_length'] = acc_clean['emp_length'].str.replace(' years', '', regex=False)


# Convert to numeric type

acc_clean['emp_length'] = pd.to_numeric(acc_clean['emp_length'])

acc_clean['emp_length'].unique()

array([10,  6,  8,  2,  9,  0,  5, -1,  1,  4,  3,  7], dtype=int64)

## Check if columns with dtype object should be ordinal

In [30]:
# Get columns with string values

cols_w_strings = acc_clean.loc[:, acc_clean.dtypes == object].columns

acc_clean[cols_w_strings]

,grade,sub_grade,emp_title,home_ownership,verification_status,issue_d,loan_status,purpose,earliest_cr_line,initial_list_status,last_credit_pull_d,application_type,verification_status_joint,sec_app_earliest_cr_line
0,C,C4,leadman,MORTGAGE,Not Verified,Dec-2015,Fully Paid,debt_consolidation,Aug-2003,w,Mar-2019,Individual,NaN,NaN
1,B,B2,Vice President of Recruiting Operations,MORTGAGE,Not Verified,Dec-2015,Fully Paid,debt_consolidation,Jun-1990,f,Mar-2019,Individual,NaN,NaN
2,B,B1,road driver,MORTGAGE,Not Verified,Dec-2015,Fully Paid,major_purchase,Feb-1999,w,Mar-2019,Individual,NaN,NaN
3,A,A2,SERVICE MANAGER,RENT,Not Verified,Dec-2015,Fully Paid,credit_card,Apr-2002,w,Mar-2019,Individual,NaN,NaN
4,B,B5,Vendor liaison,MORTGAGE,Not Verified,Dec-2015,Fully Paid,credit_card,Nov-1994,w,Nov-2018,Individual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674044,B,B4,Safety Director,OWN,Source Verified,Oct-2016,Charged Off,credit_card,Oct-2006,f,Oct-2017,Individual,NaN,NaN
674045,B,B1,Supervisor,MORTGAGE,Not Verified,Oct-2016,Charged Off,debt_consolidation,Dec-1984,f,Mar-2019,Individual,NaN,NaN
674046,C,C1,personal investor,OWN,Not Verified,Oct-2016,Charged Off,debt_consolidation,May-2004,f,Dec-2016,Individual,NaN,NaN
674047,C,C2,Area Director,MORTGAGE,Source Verified,Oct-2016,Charged Off,home_improvement,May-2005,f,Mar-2019,Individual,NaN,NaN


In [31]:
# getting the unique values for columns that have object dtype

for col in cols_w_strings:
    print(col)
    print(acc_clean[col].unique(),'\n')


grade
['C' 'B' 'A' 'E' 'D' 'F' 'G'] 

sub_grade
['C4' 'B2' 'B1' 'A2' 'B5' 'C1' 'E3' 'C3' 'A4' 'A1' 'C2' 'C5' 'D1' 'B3'
 'D3' 'D5' 'B4' 'D2' 'A5' 'F1' 'D4' 'A3' 'E2' 'E1' 'E5' 'G3' 'E4' 'F2'
 'F4' 'F3' 'G5' 'G1' 'G2' 'F5' 'G4'] 

emp_title
['leadman' 'Vice President of Recruiting Operations' 'road driver' ...
 'dentist/owner' 'Site Acquistion Specialist' 'personal investor'] 

home_ownership
['MORTGAGE' 'RENT' 'OWN' 'ANY' 'NONE' 'OTHER'] 

verification_status
['Not Verified' 'Verified' 'Source Verified'] 

issue_d
['Dec-2015' 'Nov-2015' 'Oct-2015' 'Sep-2015' 'Aug-2015' 'Jul-2015'
 'Jun-2015' 'May-2015' 'Apr-2015' 'Mar-2015' 'Feb-2015' 'Jan-2015'
 'Mar-2018' 'Feb-2018' 'Jan-2018' 'Sep-2017' 'Aug-2017' 'Jul-2017'
 'Jun-2016' 'May-2016' 'Apr-2016' 'Sep-2018' 'Aug-2018' 'Jul-2018'
 'Jun-2017' 'May-2017' 'Apr-2017' 'Mar-2016' 'Feb-2016' 'Jan-2016'
 'Dec-2014' 'Nov-2014' 'Oct-2014' 'Sep-2014' 'Aug-2014' 'Jul-2014'
 'Jun-2014' 'May-2014' 'Apr-2014' 'Mar-2014' 'Feb-2014' 'Jan-2014'
 'Dec-2018' 

['Individual' 'Joint App'] 

verification_status_joint
[nan 'Not Verified' 'Verified' 'Source Verified'] 

sec_app_earliest_cr_line
[nan 'Feb-2007' 'Jul-2015' 'Aug-2012' 'Sep-2005' 'Jul-1996' 'Oct-2006'
 'Jan-2001' 'May-2009' 'Feb-2008' 'Aug-2007' 'Jan-2007' 'Dec-2002'
 'Dec-1996' 'Mar-2003' 'Feb-2005' 'Jan-2002' 'Aug-2006' 'Jul-2008'
 'Mar-2006' 'Nov-2000' 'Aug-1994' 'Mar-2005' 'Sep-2008' 'Jan-2003'
 'Sep-1981' 'Sep-1996' 'Jun-2005' 'Dec-1999' 'Jul-2006' 'Oct-2007'
 'Aug-2002' 'Dec-2007' 'Mar-2012' 'May-1993' 'Jul-1995' 'Dec-2001'
 'Sep-2006' 'May-2005' 'Oct-2005' 'May-2003' 'Jan-2006' 'Jan-2011'
 'May-1999' 'Jan-2004' 'Dec-2010' 'Oct-1996' 'Dec-2011' 'Aug-2004'
 'May-1997' 'Nov-2002' 'Oct-1987' 'Sep-2001' 'Jan-2000' 'Sep-1985'
 'Mar-2004' 'Oct-2001' 'Dec-2000' 'Jul-1988' 'Nov-1992' 'Nov-2005'
 'Jun-2006' 'Feb-2004' 'Sep-2002' 'Feb-1997' 'Feb-2003' 'Aug-1995'
 'Sep-2000' 'Dec-1994' 'May-1995' 'Nov-2001' 'Sep-1998' 'Nov-2010'
 'Sep-2010' 'Apr-1992' 'Jul-1997' 'Jul-2011' 'Jun-2004' 'Feb

# Home Ownership

In [32]:
# Assigning values to home_ownership status

acc_clean.loc[acc_clean['home_ownership']=='OWN', 'home_ownership'] = 3

acc_clean.loc[acc_clean['home_ownership']=='MORTGAGE', 'home_ownership'] = 2

acc_clean.loc[acc_clean['home_ownership']=='ANY', 'home_ownership'] = 2

acc_clean.loc[acc_clean['home_ownership']=='OTHER', 'home_ownership'] = 1

acc_clean.loc[acc_clean['home_ownership']=='RENT', 'home_ownership'] = 0

acc_clean.loc[acc_clean['home_ownership']=='NONE', 'home_ownership'] = 0

acc_clean['home_ownership'] = acc_clean['home_ownership'].astype(int)

# Grade

In [33]:
# Grouping E, F, and G into F

acc_clean['grouped_grade_NO_EG'] = acc_clean['grade']

acc_clean['grouped_grade_NO_EG'] = acc_clean['grouped_grade_NO_EG'].str.replace('E', 'F', regex=False)

acc_clean['grouped_grade_NO_EG'] = acc_clean['grouped_grade_NO_EG'].str.replace('G', 'F', regex=False)

In [34]:
# replacing letter grades with numbers

# get sorted list of grades
grade_list = sorted(list(acc_clean['grade'].unique()))

i = 1

for grades in grade_list:
    acc_clean.loc[acc_clean['grade']==grades, 'grade'] = i
    i+=1

acc_clean['grade'] = acc_clean['grade'].astype(int)
acc_clean['grade'].unique()

array([3, 2, 1, 5, 4, 6, 7])

In [35]:
# replacing letter grades with numbers for grouped grades

# get sorted list of grades
grade_list = sorted(list(acc_clean['grouped_grade_NO_EG'].unique()))

i = 1

for grades in grade_list:
    acc_clean.loc[acc_clean['grouped_grade_NO_EG']==grades, 'grouped_grade_NO_EG'] = i
    i+=1

acc_clean['grouped_grade_NO_EG'] = acc_clean['grouped_grade_NO_EG'].astype(int)
acc_clean['grouped_grade_NO_EG'].unique()

array([3, 2, 1, 5, 4])

## Subgrade

In [36]:
# grouping subgrades 1, 2 vs 3, 4, 5

acc_clean['grouped_subgrade_12_345'] = acc_clean['sub_grade']

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].str.replace('2',
                                                                                  '1', regex=False)

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].str.replace('3',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].str.replace('4',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].str.replace('5',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_12_345'].value_counts()

B2    124010
C2    108578
C1     82205
A2     76935
B1     72803
D2     52666
D1     48263
A1     40598
E2     24507
E1     22676
F1      8670
F2      7489
G1      2590
G2      2059
Name: grouped_subgrade_12_345, dtype: int64

In [37]:
# grouping subgrades 1, 2, 3 vs 4, 5

acc_clean['grouped_subgrade_123_45'] = acc_clean['sub_grade']

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].str.replace('2',
                                                                                  '1', regex=False)

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].str.replace('3',
                                                                                  '1', regex=False)

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].str.replace('4',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].str.replace('5',
                                                                                  '2', regex=False)

acc_clean['grouped_subgrade_123_45'].value_counts()

C1    119807
B1    113831
B2     82982
C2     70976
D1     68007
A1     59529
A2     58004
D2     32922
E1     32026
E2     15157
F1     11698
F2      4461
G1      3398
G2      1251
Name: grouped_subgrade_123_45, dtype: int64

In [38]:
# grouping Subgrades

acc_clean['grouped_subgrade'] = acc_clean['sub_grade']

acc_clean['grouped_subgrade'] = acc_clean['grouped_subgrade'].str.replace('E',
                                                                                  'F', regex=False)

acc_clean['grouped_subgrade'] = acc_clean['grouped_subgrade'].str.replace('G',
                                                                                  'F', regex=False)

In [39]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_clean['sub_grade'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_clean.loc[acc_clean['sub_grade']==subgrades, 'sub_grade'] = i
    i+=1

acc_clean['sub_grade'] = acc_clean['sub_grade'].astype(int)
acc_clean['sub_grade'].unique()

array([14,  7,  6,  2, 10, 11, 23, 13,  4,  1, 12, 15, 16,  8, 18, 20,  9,
       17,  5, 26, 19,  3, 22, 21, 25, 33, 24, 27, 29, 28, 35, 31, 32, 30,
       34])

In [40]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_clean['grouped_subgrade_12_345'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_clean.loc[acc_clean['grouped_subgrade_12_345']==subgrades, 'grouped_subgrade_12_345'] = i
    i+=1

acc_clean['grouped_subgrade_12_345'] = acc_clean['grouped_subgrade_12_345'].astype(int)
acc_clean['grouped_subgrade_12_345'].unique()

array([ 6,  3,  1,  4,  5, 10,  2,  7,  8, 11,  9, 14, 12, 13])

In [41]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_clean['grouped_subgrade_123_45'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_clean.loc[acc_clean['grouped_subgrade_123_45']==subgrades, 'grouped_subgrade_123_45'] = i
    i+=1

acc_clean['grouped_subgrade_123_45'] = acc_clean['grouped_subgrade_123_45'].astype(int)
acc_clean['grouped_subgrade_123_45'].unique()

array([ 6,  3,  1,  4,  5,  9,  2,  7,  8, 11, 10, 13, 12, 14])

In [42]:
# replacing subgrades with numbers

# get sorted list of subgrades
subgrade_list = sorted(list(acc_clean['grouped_subgrade'].unique()))

i = 1

for subgrades in subgrade_list:
    acc_clean.loc[acc_clean['grouped_subgrade']==subgrades, 'grouped_subgrade'] = i
    i+=1

acc_clean['grouped_subgrade'] = acc_clean['grouped_subgrade'].astype(int)
acc_clean['grouped_subgrade'].unique()

array([14,  7,  6,  2, 10, 11, 23, 13,  4,  1, 12, 15, 16,  8, 18, 20,  9,
       17,  5, 21, 19,  3, 22, 25, 24])

### Earliest Credit Line

In [43]:
# converting the column to datetime format

acc_clean['earliest_cr_line_dt'] = pd.DatetimeIndex(acc_clean['earliest_cr_line'])
acc_clean['earliest_cr_line_dt']

0        2003-08-01
1        1990-06-01
2        1999-02-01
3        2002-04-01
4        1994-11-01
            ...    
674044   2006-10-01
674045   1984-12-01
674046   2004-05-01
674047   2005-05-01
674048   1999-07-01
Name: earliest_cr_line_dt, Length: 674049, dtype: datetime64[ns]

In [44]:
# creating a new column that will show the number of months that credit has existed for customer

acc_clean['duration_of_credit_months'] =  (pd.DatetimeIndex(acc_clean['issue_d']).year -\
                                               pd.DatetimeIndex(acc_clean['earliest_cr_line']).year)*12 +\
(pd.DatetimeIndex(acc_clean['issue_d']).month - pd.DatetimeIndex(acc_clean['earliest_cr_line']).month)

acc_clean[['duration_of_credit_months']]

,duration_of_credit_months
0,148.0
1,306.0
2,202.0
3,164.0
4,253.0
...,...
674044,120.0
674045,382.0
674046,149.0
674047,137.0


In [45]:
# dropping the columns that helped create the new column

acc_clean = acc_clean.drop(['earliest_cr_line_dt', 'earliest_cr_line'], axis=1)

### Last Credit Pulled Date

In [46]:
# dropping the column bc it is used post-funding

acc_clean = acc_clean.drop(['last_credit_pull_d'], axis=1)

# Fico Score

In [47]:
acc_clean['fico_score'] = (acc_clean['fico_range_high'] + acc_clean['fico_range_low'])/2

In [48]:
acc_clean['sec_app_fico_score'] = (acc_clean['sec_app_fico_range_high'] +\
                                   acc_clean['sec_app_fico_range_low'])/2

In [49]:
acc_clean = acc_clean.drop(['fico_range_high', 'fico_range_low',
                                    'sec_app_fico_range_high',
                                    'sec_app_fico_range_low'], axis=1)

### Verification status

In [50]:
acc_clean['verification_status'].unique()

array(['Not Verified', 'Verified', 'Source Verified'], dtype=object)

In [51]:
# if the income is verified, change to 1. If not verified, change to 0

acc_clean.loc[((acc_clean['verification_status']=='Source Verified') |\
                  (acc_clean['verification_status']=='Verified')), 'verification_status'] = 1

acc_clean.loc[acc_clean['verification_status']=='Not Verified', 'verification_status'] = 0

# change dtype to int
acc_clean['verification_status'] = acc_clean['verification_status'].astype(int)

acc_clean['verification_status']

0         0
1         0
2         0
3         0
4         0
         ..
674044    1
674045    0
674046    0
674047    1
674048    0
Name: verification_status, Length: 674049, dtype: int32

In [52]:
acc_clean['application_type'].value_counts()

Individual    661071
Joint App      12978
Name: application_type, dtype: int64

In [53]:
list_sec_app = list(acc_clean.filter(like='sec_app').columns)
list_sec_app.append('annual_inc_joint')
list_sec_app

['sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol_util',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_mths_since_last_major_derog',
 'sec_app_fico_score',
 'annual_inc_joint']

In [54]:
acc_clean[(acc_clean['sec_app_fico_score'].notnull())|
              (acc_clean['sec_app_earliest_cr_line'].notnull())|
              (acc_clean['sec_app_inq_last_6mths'].notnull())|
              (acc_clean['sec_app_mort_acc'].notnull())|
              (acc_clean['sec_app_open_acc'].notnull())|
              (acc_clean['sec_app_revol_util'].notnull())|
              (acc_clean['sec_app_open_act_il'].notnull())|
              (acc_clean['sec_app_num_rev_accts'].notnull())|
              (acc_clean['sec_app_mths_since_last_major_derog'].notnull())|
              (acc_clean['annual_inc_joint'].notnull())]


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score
237,15000.0,15000.0,15000.0,36,14.85,518.88,3,15,NaN,-1,2,37534.44,1,Dec-2015,Fully Paid,debt_consolidation,31.44,0.0,NaN,NaN,13.0,0.0,39981.0,45.0,28.0,w,18710.562289,0.0,NaN,1.0,Joint App,61366.44,19.22,Not Verified,0.0,0.0,181626.0,0.0,29.0,0.0,NaN,6579.0,45.0,88900.0,2.0,0.0,8.0,13971.0,31798.0,48.0,0.0,0.0,60.0,301.0,6.0,6.0,4.0,9.0,NaN,6.0,NaN,7.0,11.0,7.0,13.0,2.0,12.0,22.0,11.0,13.0,100.0,14.3,0.0,0.0,233900.0,39981.0,61200.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10500.0,18679.68,13075.78,3,6,6,15,301.0,682.0,NaN
342,35000.0,35000.0,35000.0,60,17.97,888.20,4,19,IT Analyst,10,2,150000.00,0,Dec-2015,Fully Paid,debt_consolidation,25.11,1.0,20.0,NaN,27.0,0.0,39913.0,85.3,45.0,w,43193.305750,0.0,25.0,1.0,Joint App,192000.00,25.28,Not Verified,0.0,433.0,603735.0,5.0,11.0,139765.0,74.0,13130.0,78.0,46800.0,3.0,8.0,5.0,22361.0,3761.0,89.4,0.0,0.0,165.0,404.0,0.0,0.0,7.0,29.0,20.0,0.0,20.0,9.0,15.0,10.0,15.0,9.0,18.0,27.0,16.0,27.0,84.4,80.0,0.0,0.0,733890.0,180363.0,35600.0,166148.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24500.0,53292.00,37304.40,4,8,8,19,404.0,672.0,NaN
874,25000.0,25000.0,25000.0,60,18.99,648.38,5,21,Service Operations Manager,10,2,37856.00,1,Dec-2015,Fully Paid,debt_consolidation,23.17,0.0,32.0,27.0,6.0,2.0,9624.0,89.1,13.0,w,34173.848630,0.0,32.0,1.0,Joint App,74684.00,11.74,Not Verified,0.0,109.0,146727.0,1.0,17.0,20709.0,83.0,7631.0,85.0,10800.0,1.0,0.0,1.0,24455.0,176.0,98.2,0.0,0.0,153.0,231.0,25.0,17.0,3.0,72.0,NaN,0.0,NaN,2.0,2.0,3.0,4.0,3.0,4.0,7.0,2.0,6.0,84.6,66.7,0.0,0.0,211261.0,30333.0,9800.0,24961.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17500.0,38902.80,27231.96,5,9,9,21,231.0,672.0,NaN
893,10500.0,10500.0,10500.0,36,9.80,337.83,2,8,Supervisor,0,2,65000.00,1,Dec-2015,Fully Paid,debt_consolidation,21.49,1.0,10.0,NaN,29.0,0.0,11156.0,83.9,43.0,w,12150.121933,0.0,65.0,1.0,Joint App,92000.00,15.85,Not Verified,0.0,0.0,350643.0,24.0,1.0,167875.0,96.0,4970.0,95.0,13300.0,0.0,1.0,7.0,12091.0,2144.0,83.9,0.0,0.0,127.0,287.0,17.0,1.0,1.0,17.0,10.0,2.0,10.0,4.0,4.0,4.0,6.0,36.0,4.0,6.0,4.0,29.0,93.0,50.0,0.0,0.0,373179.0,179031.0,13300.0,184122.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7350.0,12161.88,8513.32,2,4,3,8,287.0,667.0,NaN
960,26600.0,26600.0,26600.0,60,19.48,697.07,5,22,Postmaster,9,2,65000.00,1,Dec-2015,Fully Paid,debt_consolidation,51.20,0.0,NaN,NaN,18.0,0.0,36896.0,94.1,43.0,w,34204.374736,0.0,NaN,1.0,Joint App,120000.00,28.45,Not Verified,0.0,0.0,154550.0,5.0,13.0,63469.0,46.0,6887.0,63.0,39200.0,0.0,4.

In [55]:
acc_clean.loc[acc_clean['application_type']=='Joint App', 'application_type'] = 2
acc_clean.loc[acc_clean['application_type']=='Individual', 'application_type'] = 1

acc_clean['application_type'] = acc_clean['application_type'].astype(int)

In [56]:
acc_clean['verification_status_joint'].unique()

array([nan, 'Not Verified', 'Verified', 'Source Verified'], dtype=object)

In [57]:
'''
for verification_status_joint where there are values in cols that start with
sec_app, but verification_status_joint is null
'''

# add in 'not verified' where applicable

acc_clean.loc[((acc_clean['verification_status_joint'].notnull()) &\
                   (acc_clean['application_type']==1)),
                  'verification_status_joint'] = 'Not Verified'

In [58]:
acc_clean[((acc_clean['verification_status_joint'].isnull()) &\
                  (acc_clean['application_type']==2))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score
152595,30000.0,30000.0,30000.0,36,5.31,903.31,1,1,Associate Teacher,2,2,20000.0,1,Mar-2018,Fully Paid,debt_consolidation,43.04,0.0,NaN,NaN,10.0,0.0,35332.0,25.1,17.0,w,31021.510161,0.0,NaN,1.0,2,68000.0,12.65,NaN,0.0,0.0,35332.0,0.0,23.0,0.0,NaN,17531.0,25.0,140700.0,0.0,0.0,2.0,3926.0,103868.0,25.4,0.0,0.0,136.0,204.0,16.0,16.0,0.0,16.0,NaN,23.0,NaN,3.0,3.0,8.0,9.0,2.0,10.0,15.0,3.0,10.0,100.0,12.5,0.0,0.0,140700.0,35332.0,139200.0,0.0,35332.0,Jan-2003,0.0,4.0,8.0,28.0,0.0,7.0,NaN,21000.0,32519.16,22763.41,1,1,1,1,204.0,757.0,747.0
153211,14500.0,14500.0,14500.0,36,5.31,436.60,1,1,Software Developer,2,2,74000.0,1,Mar-2018,Fully Paid,debt_consolidation,9.22,0.0,NaN,NaN,5.0,0.0,16257.0,45.2,7.0,w,15116.591168,0.0,NaN,1.0,2,136000.0,9.46,NaN,0.0,0.0,176952.0,1.0,6.0,3486.0,48.0,14095.0,46.0,36000.0,1.0,0.0,4.0,35390.0,18092.0,45.2,0.0,0.0,79.0,261.0,3.0,3.0,1.0,10.0,NaN,0.0,NaN,2.0,3.0,2.0,2.0,2.0,3.0,4.0,3.0,5.0,100.0,0.0,0.0,0.0,200825.0,19743.0,33000.0,7200.0,38271.0,Sep-2010,1.0,1.0,13.0,74.4,2.0,11.0,NaN,10150.0,15717.60,11002.32,1,1,1,1,261.0,742.0,622.0
153237,35000.0,35000.0,35000.0,60,18.45,897.37,4,17,Office Specialist III,2,2,65000.0,1,Mar-2018,Fully Paid,debt_consolidation,17.43,0.0,NaN,NaN,10.0,0.0,32693.0,74.5,12.0,w,40066.441011,0.0,NaN,1.0,2,148000.0,13.64,NaN,0.0,0.0,403920.0,0.0,NaN,0.0,NaN,16322.0,75.0,43900.0,1.0,0.0,5.0,40392.0,5384.0,85.2,0.0,0.0,NaN,76.0,11.0,1.0,3.0,12.0,NaN,0.0,NaN,6.0,9.0,6.0,6.0,0.0,9.0,9.0,9.0,10.0,100.0,66.7,0.0,0.0,415127.0,32693.0,36400.0,0.0,34704.0,Nov-2007,0.0,3.0,9.0,77.7,1.0,9.0,NaN,24500.0,53842.20,37689.54,4,7,7,17,76.0,672.0,677.0
153427,7500.0,7500.0,7500.0,36,6.07,228.41,1,2,Technician,2,2,39500.0,0,Mar-2018,Fully Paid,debt_consolidation,18.05,0.0,NaN,NaN,9.0,0.0,6339.0,40.9,19.0,w,7655.906205,0.0,NaN,1.0,2,61500.0,12.57,NaN,0.0,0.0,118767.0,1.0,26.0,3472.0,35.0,3363.0,39.0,15500.0,1.0,0.0,1.0,13196.0,3796.0,56.9,0.0,0.0,95.0,96.0,8.0,8.0,1.0,36.0,NaN,0.0,NaN,3.0,5.0,4.0,4.0,7.0,7.0,11.0,5.0,9.0,100.0,25.0,0.0,0.0,142846.0,9811.0,8800.0,10000.0,10625.0,Feb-2010,0.0,0.0,14.0,46.4,9.0,11.0,NaN,5250.0,8222.76,5755.93,1,1,1,2,96.0,712.0,707.0
153473,23000.0,23000.0,23000.0,36,5.31,692.54,1,1,Registered Nurse,1,2,72000.0,0,Mar-2018,Fully Paid,debt_consolidation,33.08,0.0,66.0,NaN,17.0,0.0,21565.0,36.9,52.0,w,23491.672945,0.0,NaN,1.0,2,217000.0,10.98,NaN,0.0,0.0,258968.0,4.0,21.0,105227.0,47.0,14029.0,41.0,58500.0,0.0,6.0,3.0,19921.0,27435.0,44.0,0.0,0.0,185.0,163.0,9.0,9.0,5.0,9.0,NaN

In [59]:
acc_clean.loc[((acc_clean['verification_status_joint'].isnull()) &\
                   (acc_clean['application_type']==2)),
                  'verification_status_joint'] = 'Not Verified'

In [60]:
acc_clean[((acc_clean['verification_status_joint'].notnull()) &\
                  (acc_clean['application_type']==1))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score


In [61]:
acc_clean[((acc_clean['verification_status_joint'].isnull()) &\
               (acc_clean['application_type']==1))]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,leadman,10,2,55000.0,0,Dec-2015,Fully Paid,debt_consolidation,5.91,0.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,0.0,30.0,1.0,1,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2520.0,4429.08,3100.36,3,6,6,14,148.0,677.0,NaN
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,Vice President of Recruiting Operations,10,2,180000.0,0,Dec-2015,Fully Paid,debt_consolidation,14.67,0.0,49.0,NaN,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,0.0,NaN,1.0,1,NaN,NaN,NaN,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,NaN,10.0,NaN,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14000.0,22952.88,16067.02,2,3,3,7,306.0,682.0,NaN
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,road driver,10,2,85000.0,0,Dec-2015,Fully Paid,major_purchase,17.61,1.0,3.0,NaN,8.0,0.0,826.0,5.7,15.0,w,21538.508977,0.0,3.0,1.0,1,NaN,NaN,NaN,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,NaN,8.0,NaN,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14000.0,22725.36,15907.75,2,3,3,6,202.0,707.0,NaN
3,10000.0,10000.0,10000.0,36,6.49,306.45,1,2,SERVICE MANAGER,6,0,85000.0,0,Dec-2015,Fully Paid,credit_card,13.07,0.0,NaN,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,0.0,NaN,1.0,1,NaN,NaN,NaN,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,NaN,1.0,NaN,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7000.0,11032.20,7722.54,1,1,1,2,164.0,687.0,NaN
4,8000.0,8000.0,8000.0,36,11.48,263.74,2,10,Vendor liaison,10,2,42000.0,0,Dec-2015,Fully Paid,credit_card,34.80,0.0,75.0,NaN,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,0.0,75.0,1.0,1,NaN,NaN,NaN,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,NaN,10.0,NaN,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0

In [62]:
acc_clean.loc[((acc_clean['verification_status_joint'].isnull()) &\
                   (acc_clean['application_type']==1)),
                  'verification_status_joint'] = 'Individual'

In [63]:
# if the income is verified, change to 1. If not verified, change to 0

acc_clean.loc[((acc_clean['verification_status_joint']=='Source Verified') |\
                  (acc_clean['verification_status_joint']=='Verified')),
                  'verification_status_joint'] = 1

acc_clean.loc[acc_clean['verification_status_joint']=='Individual',
                  'verification_status_joint'] = -1

acc_clean.loc[acc_clean['verification_status_joint']=='Not Verified',
                  'verification_status_joint'] = 0

In [64]:
acc_clean['verification_status_joint'] = acc_clean['verification_status_joint'].astype(int)

## Reducing Income if not Verified

In [65]:
acc_clean['annual_inc_reduced'] = acc_clean['annual_inc']

acc_clean.loc[acc_clean['verification_status']==0,
                  'annual_inc_reduced'] = acc_clean['annual_inc'] * 0.7

In [66]:
acc_clean['annual_inc_joint_reduced'] = acc_clean['annual_inc_joint']

acc_clean.loc[acc_clean['verification_status_joint']==0,
                  'annual_inc_joint_reduced'] = acc_clean['annual_inc_joint'] * 0.7

### Joint Earliest Credit Line

In [67]:
# converting column to datetime

acc_clean['sec_app_earliest_cr_line_dt'] = pd.DatetimeIndex(acc_clean['sec_app_earliest_cr_line'])
acc_clean['sec_app_earliest_cr_line_dt']

0        NaT
1        NaT
2        NaT
3        NaT
4        NaT
          ..
674044   NaT
674045   NaT
674046   NaT
674047   NaT
674048   NaT
Name: sec_app_earliest_cr_line_dt, Length: 674049, dtype: datetime64[ns]

In [68]:
# checking how many non-null values there are

acc_clean[acc_clean['sec_app_earliest_cr_line'].notnull()]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_earliest_cr_line_dt
152446,1000.0,1000.0,1000.0,36,17.47,35.89,4,16,Lead Sanitation Operator,2,0,51000.0,1,Mar-2018,Fully Paid,credit_card,10.56,0.0,61.0,103.0,5.0,1.0,2.0,0.1,9.0,w,1032.903619,0.0,61.0,1.0,2,122000.0,7.47,1,0.0,0.0,22707.0,3.0,17.0,22705.0,95.0,2.0,86.0,2500.0,4.0,0.0,3.0,4541.0,2498.0,0.1,0.0,0.0,120.0,9.0,1.0,1.0,0.0,1.0,NaN,1.0,NaN,1.0,1.0,2.0,2.0,7.0,2.0,2.0,1.0,5.0,88.9,0.0,0.0,1.0,26501.0,22707.0,2500.0,24001.0,1282.0,Feb-2007,5.0,1.0,13.0,32.0,9.0,12.0,62.0,700.0,1292.04,904.43,4,7,7,16,120.0,707.0,637.0,51000.0,122000.0,2007-02-01
152462,5750.0,5750.0,5725.0,36,13.58,195.36,3,12,Housekeeing,1,0,1200.0,1,Mar-2018,Fully Paid,small_business,63.00,0.0,NaN,NaN,12.0,0.0,2408.0,12.4,13.0,f,6346.533647,0.0,NaN,1.0,2,18000.0,4.47,1,0.0,0.0,2408.0,0.0,NaN,0.0,NaN,1875.0,12.0,19400.0,0.0,0.0,11.0,201.0,6925.0,21.3,0.0,0.0,NaN,37.0,4.0,4.0,0.0,4.0,NaN,15.0,NaN,1.0,2.0,4.0,4.0,0.0,12.0,13.0,2.0,12.0,100.0,0.0,0.0,0.0,19400.0,2408.0,8800.0,0.0,2412.0,Jul-2015,0.0,0.0,7.0,0.0,0.0,8.0,NaN,4025.0,7032.96,4923.07,3,5,5,12,37.0,727.0,667.0,1200.0,18000.0,2015-07-01
152476,10000.0,10000.0,10000.0,60,9.43,209.68,2,6,Quality ambassador,10,2,65000.0,0,Mar-2018,Fully Paid,home_improvement,10.97,0.0,NaN,NaN,3.0,0.0,586.0,13.0,6.0,w,10602.905501,0.0,NaN,1.0,2,115000.0,7.64,0,0.0,0.0,182306.0,1.0,24.0,29405.0,NaN,586.0,13.0,4500.0,0.0,0.0,2.0,60769.0,3914.0,13.0,0.0,0.0,88.0,37.0,12.0,12.0,1.0,12.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,3.0,100.0,0.0,0.0,0.0,206180.0,29991.0,4500.0,38280.0,2619.0,Aug-2012,1.0,1.0,4.0,26.8,1.0,2.0,NaN,7000.0,12580.80,8806.56,2,3,3,6,88.0,792.0,752.0,45500.0,80500.0,2012-08-01
152479,38750.0,38750.0,38750.0,60,10.90,840.59,2,9,Senior Ageny,4,2,70000.0,1,Mar-2018,Fully Paid,home_improvement,21.22,0.0,NaN,NaN,10.0,0.0,4751.0,26.8,65.0,w,40144.996528,0.0,NaN,1.0,2,180000.0,12.59,1,0.0,4139.0,310771.0,3.0,5.0,106090.0,62.0,1518.0,52.0,17700.0,0.0,0.0,4.0,34530.0,9782.0,13.4,0.0,0.0,179.0,150.0,7.0,5.0,3.0,7.0,NaN,2.0,NaN,1.0,3.0,3.0,5.0,48.0,6.0,14.0,3.0,10.0,100.0,0.0,0.0,0.0,348051.0,110841.0,11300.0,124155.0,29919.0,Sep-2005,1.0,3.0,29.0,26.5,15.0,21.0,NaN,27125.0,50435.40,35304.78,2,4,4,9,179.0,717.0,737.0,70000.0,180000.0,2005-09-01
152481,27900.0,27900.0,27900.0,36,23.87,1092.70,5,22,Territory Account Manager,10,2,90000.0,1,Mar-2018,Fully Paid,credit_card,49.96,0.0,37.0,NaN,12.0,0.0,63471.0,94.5,23.0,f,31320.614837,0.0,NaN,1.0,2,170000.0,38.29,1,

In [69]:
# confirming that we can use 0 as a plausible value for null values

acc_clean[acc_clean['sec_app_earliest_cr_line']<=acc_clean['issue_d']]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_earliest_cr_line_dt
152446,1000.0,1000.0,1000.0,36,17.47,35.89,4,16,Lead Sanitation Operator,2,0,51000.0,1,Mar-2018,Fully Paid,credit_card,10.56,0.0,61.0,103.0,5.0,1.0,2.0,0.1,9.0,w,1032.903619,0.0,61.0,1.0,2,122000.0,7.47,1,0.0,0.0,22707.0,3.0,17.0,22705.0,95.0,2.0,86.0,2500.0,4.0,0.0,3.0,4541.0,2498.0,0.1,0.0,0.0,120.0,9.0,1.0,1.0,0.0,1.0,NaN,1.0,NaN,1.0,1.0,2.0,2.0,7.0,2.0,2.0,1.0,5.0,88.9,0.0,0.0,1.0,26501.0,22707.0,2500.0,24001.0,1282.0,Feb-2007,5.0,1.0,13.0,32.0,9.0,12.0,62.0,700.0,1292.04,904.43,4,7,7,16,120.0,707.0,637.0,51000.0,122000.0,2007-02-01
152462,5750.0,5750.0,5725.0,36,13.58,195.36,3,12,Housekeeing,1,0,1200.0,1,Mar-2018,Fully Paid,small_business,63.00,0.0,NaN,NaN,12.0,0.0,2408.0,12.4,13.0,f,6346.533647,0.0,NaN,1.0,2,18000.0,4.47,1,0.0,0.0,2408.0,0.0,NaN,0.0,NaN,1875.0,12.0,19400.0,0.0,0.0,11.0,201.0,6925.0,21.3,0.0,0.0,NaN,37.0,4.0,4.0,0.0,4.0,NaN,15.0,NaN,1.0,2.0,4.0,4.0,0.0,12.0,13.0,2.0,12.0,100.0,0.0,0.0,0.0,19400.0,2408.0,8800.0,0.0,2412.0,Jul-2015,0.0,0.0,7.0,0.0,0.0,8.0,NaN,4025.0,7032.96,4923.07,3,5,5,12,37.0,727.0,667.0,1200.0,18000.0,2015-07-01
152476,10000.0,10000.0,10000.0,60,9.43,209.68,2,6,Quality ambassador,10,2,65000.0,0,Mar-2018,Fully Paid,home_improvement,10.97,0.0,NaN,NaN,3.0,0.0,586.0,13.0,6.0,w,10602.905501,0.0,NaN,1.0,2,115000.0,7.64,0,0.0,0.0,182306.0,1.0,24.0,29405.0,NaN,586.0,13.0,4500.0,0.0,0.0,2.0,60769.0,3914.0,13.0,0.0,0.0,88.0,37.0,12.0,12.0,1.0,12.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,3.0,1.0,2.0,1.0,3.0,100.0,0.0,0.0,0.0,206180.0,29991.0,4500.0,38280.0,2619.0,Aug-2012,1.0,1.0,4.0,26.8,1.0,2.0,NaN,7000.0,12580.80,8806.56,2,3,3,6,88.0,792.0,752.0,45500.0,80500.0,2012-08-01
152481,27900.0,27900.0,27900.0,36,23.87,1092.70,5,22,Territory Account Manager,10,2,90000.0,1,Mar-2018,Fully Paid,credit_card,49.96,0.0,37.0,NaN,12.0,0.0,63471.0,94.5,23.0,f,31320.614837,0.0,NaN,1.0,2,170000.0,38.29,1,0.0,0.0,260181.0,3.0,21.0,49364.0,52.0,27106.0,69.0,67200.0,2.0,5.0,2.0,21682.0,879.0,98.3,0.0,0.0,123.0,249.0,36.0,6.0,3.0,36.0,NaN,7.0,37.0,3.0,4.0,3.0,3.0,10.0,8.0,10.0,4.0,12.0,95.7,100.0,0.0,0.0,312604.0,112835.0,50900.0,95404.0,145386.0,Jul-1996,0.0,3.0,9.0,84.7,1.0,10.0,NaN,19530.0,39337.20,27536.04,5,9,9,22,249.0,727.0,687.0,90000.0,170000.0,1996-07-01
152486,30000.0,30000.0,30000.0,36,6.07,913.61,1,2,Community Liaison,3,2,150000.0,1,Mar-2018,Fully Paid,debt_consolidation,23.40,0.0,NaN,NaN,15.0,0.0,20979.0,20.0,39.0,w,30406.296711,0.0,NaN,1.0,2,230000.0,

In [70]:
# creating a new column that will show the number of months that credit has existed for joint customer

acc_clean['sec_app_duration_of_credit_months'] = (pd.DatetimeIndex(acc_clean['issue_d']).year\
                                                      - pd.DatetimeIndex(acc_clean['sec_app_earliest_cr_line']).year)*12 +\
(pd.DatetimeIndex(acc_clean['issue_d']).month - pd.DatetimeIndex(acc_clean['sec_app_earliest_cr_line']).month)

# converting null values to -1
acc_clean.loc[acc_clean['sec_app_duration_of_credit_months'].isnull(),
                  'sec_app_duration_of_credit_months'] = -1

acc_clean['sec_app_duration_of_credit_months']

0        -1.0
1        -1.0
2        -1.0
3        -1.0
4        -1.0
         ... 
674044   -1.0
674045   -1.0
674046   -1.0
674047   -1.0
674048   -1.0
Name: sec_app_duration_of_credit_months, Length: 674049, dtype: float64

In [71]:
acc_clean = acc_clean.drop(['sec_app_earliest_cr_line_dt', 'sec_app_earliest_cr_line',
                                    'issue_d', 
                                    'sec_app_earliest_cr_line',
                                    'sec_app_earliest_cr_line_dt'], axis=1)

## Check the max values of columns starting with 'mths_since_'

In [72]:
cols_w_mths_since = [col for col in acc_clean.columns if col.startswith('mths_since_')]
acc_clean[cols_w_mths_since]

,mths_since_last_delinq,mths_since_last_record,mths_since_last_major_derog,mths_since_rcnt_il,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq
0,30.0,NaN,30.0,21.0,4.0,69.0,4.0,69.0
1,49.0,NaN,NaN,18.0,12.0,NaN,10.0,NaN
2,3.0,NaN,3.0,13.0,32.0,NaN,8.0,NaN
3,NaN,106.0,NaN,35.0,4.0,NaN,1.0,NaN
4,75.0,NaN,75.0,10.0,50.0,NaN,10.0,NaN
...,...,...,...,...,...,...,...,...
674044,NaN,64.0,NaN,11.0,45.0,NaN,2.0,NaN
674045,NaN,NaN,NaN,2.0,6.0,NaN,2.0,NaN
674046,62.0,66.0,62.0,13.0,69.0,62.0,8.0,62.0
674047,55.0,NaN,55.0,16.0,0.0,55.0,NaN,55.0


In [73]:
acc_clean[cols_w_mths_since].max()

mths_since_last_delinq            202.0
mths_since_last_record            123.0
mths_since_last_major_derog       202.0
mths_since_rcnt_il                488.0
mths_since_recent_bc              610.0
mths_since_recent_bc_dlq          202.0
mths_since_recent_inq              25.0
mths_since_recent_revol_delinq    202.0
dtype: float64

In [74]:
for col in cols_w_mths_since:
    acc_clean.loc[acc_clean[col].isnull(), col] = 10000

In [75]:
acc_clean[cols_w_mths_since].max()

mths_since_last_delinq            10000.0
mths_since_last_record            10000.0
mths_since_last_major_derog       10000.0
mths_since_rcnt_il                10000.0
mths_since_recent_bc              10000.0
mths_since_recent_bc_dlq          10000.0
mths_since_recent_inq             10000.0
mths_since_recent_revol_delinq    10000.0
dtype: float64

## Get cols with null values

In [76]:
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_clean, cols_w_null)

acc_clean[cols_w_null]

,emp_title,annual_inc,dti,delinq_2yrs,open_acc,pub_rec,revol_util,total_acc,collections_12_mths_ex_med,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced
0,leadman,55000.0,5.91,0.0,7.0,0.0,29.7,13.0,0.0,NaN,NaN,0.0,722.0,144904.0,2.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.0,NaN,38500.0,NaN
1,Vice President of Recruiting Operations,180000.0,14.67,0.0,12.0,0.0,84.5,27.0,0.0,NaN,NaN,0.0,0.0,360358.0,2.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306.0,NaN,126000.0,NaN
2,road driver,85000.0,17.61,1.0,8.0,0.0,5.7,15.0,0.0,NaN,NaN,0.0,0.0,141601.0,3.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.0,NaN,59500.0,NaN
3,SERVICE MANAGER,85000.0,13.07,0.0,14.0,1.0,34.5,23.0,0.0,NaN,NaN,0.0,8341.0,27957.0,1.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164.0,NaN,59500.0,NaN
4,Vendor liaison,42000.0,34.80,0.0,8.0,0.0,39.1,18.0,0.0,NaN,NaN,0.0,0.0,199696.0,2.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253.0,NaN,29400.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674044,Safety Director,49400.0,13.85,0.0,18.0,1.0,35.2,28.0,0.0,NaN,NaN,0.0,0.0,21772.0,1.0,15993.0,88.0,2151.0,63.0,16400.0,0.0,1.0,12.0,1361.0,757.0,76.3,0.0,0.0,110.0,119.0,6.0,6.0,0.0,2.0,8.0,2.0,8.0,4.0,17.0,24.0,8.0,18.0,100.0,50.0,1.0,0.0,34605.0,21772.0,3200.0,18205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,NaN,49400.0,NaN
674045,Supervisor,75000.0,15.01,0.0,12.0,0.0,44.7,25.0,0.0,NaN,NaN,0.0,4695.0,112063.0,2.0,29841.0,87.0,3716.0,73.0,16400.0,0.0,1.0,6.0,9339.0,578.0,91.5,0.0,0.0,151.0,381.0,1.0,1.0,5.0,2.0,5.0,2.0,2.0,6.0,9.0,14.0,5.0,12.0,96.0,100.0,0.0,0.0,163456.0,37171.0,6800.0,34256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,382.0,NaN,52500.0,NaN
674046,personal investor,131209.0,9.65,0.0,15.0,2.0,50.8,25.0,0.0,NaN,NaN,0.0,0.0,112459.0,3.0,94226.0,85.0,1386.0,66.0,35900.0,0.0,9.0,5.0,7497.0,3019.0,41.9,0.0,0.0,148.0,148.0,2.0,2.0,0.0,2.0,9.0,3.0,6.0,6.0,12.0,19.0,9.0,15.0,92.0,0.0,0.0,2.0,112707.0,112459.0,5200.0,76807.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.0,NaN,91846.3,NaN
674047,Area Director,120000.0,14.44,0.0,14.0,0.0,37.8,27.0,0.0,NaN,NaN,0.0,0.0,259919.0,1.0,1849.0,13.0,13424.0,34.0,83400.0,0.0,0.0,4.0,18566.0,42250

In [77]:
cols_w_null

['emp_title',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'open_acc',
 'pub_rec',
 'revol_util',
 'total_acc',
 'collections_12_mths_ex_med',
 'annual_inc_joint',
 'dti_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_act_il',
 'total_bal_il',
 'il_util',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'revol_bal_joint',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_revol

## Fill in null values

In [78]:
# creates a list of the index where the column == 0

list(acc_clean[acc_clean['emp_length']==0].index)

[12,
 27,
 42,
 44,
 48,
 54,
 89,
 95,
 100,
 112,
 114,
 124,
 128,
 131,
 134,
 138,
 139,
 140,
 142,
 160,
 163,
 164,
 166,
 168,
 178,
 179,
 187,
 206,
 219,
 246,
 259,
 261,
 286,
 294,
 295,
 317,
 322,
 343,
 370,
 378,
 416,
 431,
 435,
 436,
 442,
 452,
 457,
 461,
 465,
 474,
 476,
 477,
 486,
 511,
 513,
 542,
 590,
 591,
 602,
 605,
 611,
 612,
 615,
 641,
 658,
 663,
 665,
 668,
 673,
 694,
 710,
 724,
 727,
 728,
 729,
 736,
 772,
 808,
 817,
 830,
 833,
 850,
 868,
 893,
 897,
 914,
 924,
 928,
 938,
 939,
 968,
 977,
 1008,
 1011,
 1015,
 1016,
 1032,
 1034,
 1050,
 1056,
 1066,
 1081,
 1083,
 1087,
 1091,
 1101,
 1107,
 1116,
 1122,
 1137,
 1144,
 1145,
 1153,
 1156,
 1158,
 1162,
 1180,
 1183,
 1186,
 1215,
 1221,
 1223,
 1244,
 1248,
 1253,
 1264,
 1265,
 1278,
 1304,
 1310,
 1336,
 1338,
 1376,
 1383,
 1384,
 1388,
 1396,
 1401,
 1423,
 1434,
 1457,
 1475,
 1485,
 1489,
 1492,
 1500,
 1508,
 1547,
 1566,
 1574,
 1600,
 1616,
 1626,
 1627,
 1628,
 1720,
 1731,
 

In [79]:
# getting the columns where there are 0s

cols_w_0s_and_null = []

for col in cols_w_null:
    
    # if the list is not empty, then there are values == 0
    if list(acc_clean[acc_clean[col]==0].index)!=[]:
        cols_w_0s_and_null.append(col)
    
    for col_0 in cols_w_0s_and_null:
        acc_clean.loc[acc_clean[col_0].isnull(), col_0] = -1
    
acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,leadman,10,2,55000.0,0,Fully Paid,debt_consolidation,5.91,0.0,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,0.0,30.0,1.0,1,NaN,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2520.0,4429.08,3100.36,3,6,6,14,148.0,677.0,NaN,38500.0,NaN,-1.0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,Vice President of Recruiting Operations,10,2,180000.0,0,Fully Paid,debt_consolidation,14.67,0.0,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,0.0,10000.0,1.0,1,NaN,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14000.0,22952.88,16067.02,2,3,3,7,306.0,682.0,NaN,126000.0,NaN,-1.0
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,road driver,10,2,85000.0,0,Fully Paid,major_purchase,17.61,1.0,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,w,21538.508977,0.0,3.0,1.0,1,NaN,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14000.0,22725.36,15907.75,2,3,3,6,202.0,707.0,NaN,59500.0,NaN,-1.0
3,10000.0,10000.0,10000.0,36,6.49,306.45,1,2,SERVICE MANAGER,6,0,85000.0,0,Fully Paid,credit_card,13.07,0.0,10000.0,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,0.0,10000.0,1.0,1,NaN,-1.0,-1,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,10000.0,1.0,10000.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7000.0,11032.20,7722.54,1,1,1,2,164.0,687.0,NaN,59500.0,NaN,-1.0
4,8000.0,8000.0,8000.0,36,11.48,263.74,2,10,Vendor liaison,10,2,42000.0,0,Fully Paid,credit_card,34.80,0.0,75.0,10000.0,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,0.0,75.0,1.0,1,NaN,-1.0,-1,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,

In [80]:
# rerun to get new cols w null values

cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_clean, cols_w_null)

acc_clean[cols_w_null]

,emp_title,total_acc,annual_inc_joint,mo_sin_old_rev_tl_op,num_rev_accts,duration_of_credit_months,sec_app_fico_score,annual_inc_joint_reduced
0,leadman,13.0,NaN,128.0,9.0,148.0,NaN,NaN
1,Vice President of Recruiting Operations,27.0,NaN,306.0,16.0,306.0,NaN,NaN
2,road driver,15.0,NaN,55.0,3.0,202.0,NaN,NaN
3,SERVICE MANAGER,23.0,NaN,129.0,19.0,164.0,NaN,NaN
4,Vendor liaison,18.0,NaN,253.0,11.0,253.0,NaN,NaN
...,...,...,...,...,...,...,...,...
674044,Safety Director,28.0,NaN,119.0,24.0,120.0,NaN,NaN
674045,Supervisor,25.0,NaN,381.0,14.0,382.0,NaN,NaN
674046,personal investor,25.0,NaN,148.0,19.0,149.0,NaN,NaN
674047,Area Director,27.0,NaN,136.0,23.0,137.0,NaN,NaN


In [81]:
acc_clean.loc[acc_clean['emp_title'].isnull(), 'emp_title'] = 'Unknown'

### Employment length

In [82]:
emplength_mean = acc_clean['emp_length'].mean()

In [83]:
acc_clean.loc[(acc_clean['emp_length'].isnull()) &\
                  (acc_clean['emp_title'].notnull()), 'emp_length'] = emplength_mean

In [84]:
acc_clean.loc[(acc_clean['emp_length'].isnull()) &\
                  (acc_clean['emp_title'].isnull()), 'emp_length'] = -1

### Employment Title

In [85]:
acc_clean['emp_title'].value_counts()

Unknown                                 43157
Teacher                                 10682
Manager                                  9686
Owner                                    5182
Registered Nurse                         4487
                                        ...  
Flooring Technician                         1
Administrative Clerical                     1
Interdealer Information Technologies        1
Sales/Business Manager                      1
Supervisor, IT Systems                      1
Name: emp_title, Length: 216721, dtype: int64

In [86]:
acc_clean = acc_clean.drop('emp_title', axis=1)

### joint

In [87]:
# if no secondary app, convert to -1

acc_clean.loc[acc_clean['annual_inc_joint'].isnull(), 'annual_inc_joint'] = -1

acc_clean.loc[acc_clean['annual_inc_joint_reduced'].isnull(),'annual_inc_joint_reduced'] = -1

acc_clean.loc[acc_clean['sec_app_fico_score'].isnull(), 'sec_app_fico_score'] = -1

### others

In [88]:
acc_clean.loc[acc_clean['annual_inc'].isnull(), 'annual_inc'] = -1

acc_clean.loc[acc_clean['mo_sin_old_rev_tl_op'].isnull(), 'mo_sin_old_rev_tl_op'] = 10000

acc_clean.loc[acc_clean['duration_of_credit_months'].isnull(),
                  'duration_of_credit_months'] = -1

acc_clean.loc[acc_clean['sec_app_duration_of_credit_months'].isnull(),
                  'sec_app_duration_of_credit_months'] = -1

### total accounts

In [89]:
cols_w_acc = [col for col in acc_clean.columns if 'acc' in col]
acc_clean[cols_w_acc]

,open_acc,total_acc,acc_now_delinq,acc_open_past_24mths,mo_sin_old_il_acct,mort_acc,num_rev_accts,sec_app_mort_acc,sec_app_open_acc,sec_app_num_rev_accts
0,7.0,13.0,0.0,4.0,148.0,1.0,9.0,-1.0,-1.0,-1.0
1,12.0,27.0,0.0,6.0,142.0,4.0,16.0,-1.0,-1.0,-1.0
2,8.0,15.0,0.0,4.0,149.0,3.0,3.0,-1.0,-1.0,-1.0
3,14.0,23.0,0.0,7.0,164.0,1.0,19.0,-1.0,-1.0,-1.0
4,8.0,18.0,0.0,5.0,155.0,1.0,11.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
674044,18.0,28.0,0.0,12.0,110.0,0.0,24.0,-1.0,-1.0,-1.0
674045,12.0,25.0,0.0,6.0,151.0,5.0,14.0,-1.0,-1.0,-1.0
674046,15.0,25.0,0.0,5.0,148.0,0.0,19.0,-1.0,-1.0,-1.0
674047,14.0,27.0,0.0,4.0,16.0,3.0,23.0,-1.0,-1.0,-1.0


In [90]:
# checking if we can use open accounts for null values

acc_clean[(acc_clean['total_acc'].isnull()) & (acc_clean['open_acc'].isnull())][['total_acc',
                                                                                 'open_acc']]

,total_acc,open_acc


In [91]:
# using the open_acc as a filler for the null values for total_acc

acc_clean.loc[acc_clean['total_acc'].isnull(), 'total_acc'] = acc_clean['open_acc']

In [92]:
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_clean, cols_w_null)

acc_clean[cols_w_null]

,num_rev_accts
0,9.0
1,16.0
2,3.0
3,19.0
4,11.0
...,...
674044,24.0
674045,14.0
674046,19.0
674047,23.0


In [93]:
cols_w_null_list = list(acc_clean[cols_w_null].columns)
cols_w_null_list

['num_rev_accts']

In [94]:
# getting the columns where there are 0s

for col in cols_w_null_list:
    acc_clean.loc[acc_clean[col].isnull(), col] = -1
    
acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,10.0,2,55000.0,0,Fully Paid,debt_consolidation,5.91,0.0,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,0.0,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2520.0,4429.08,3100.36,3,6,6,14,148.0,677.0,-1.0,38500.0,-1.0,-1.0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,10.0,2,180000.0,0,Fully Paid,debt_consolidation,14.67,0.0,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,6.0,30030.0,0.0,102.9,0.0,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14000.0,22952.88,16067.02,2,3,3,7,306.0,682.0,-1.0,126000.0,-1.0,-1.0
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,10.0,2,85000.0,0,Fully Paid,major_purchase,17.61,1.0,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,w,21538.508977,0.0,3.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,4.0,17700.0,13674.0,5.7,0.0,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14000.0,22725.36,15907.75,2,3,3,6,202.0,707.0,-1.0,59500.0,-1.0,-1.0
3,10000.0,10000.0,10000.0,36,6.49,306.45,1,2,6.0,0,85000.0,0,Fully Paid,credit_card,13.07,0.0,10000.0,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,0.0,10000.0,1.0,1,-1.0,-1.0,-1,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,7.0,1997.0,8182.0,50.1,0.0,0.0,164.0,129.0,1.0,1.0,1.0,4.0,10000.0,1.0,10000.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7000.0,11032.20,7722.54,1,1,1,2,164.0,687.0,-1.0,59500.0,-1.0,-1.0
4,8000.0,8000.0,8000.0,36,11.48,263.74,2,10,10.0,2,42000.0,0,Fully Paid,credit_card,34.80,0.0,75.0,10000.0,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,0.0,75.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,5.0,28528.0,9966.0,41.4,0.0,0.0,155.0,253.0,15.0,10.0,1.0,50.0,10000.0,10.0,10000.0,3.0,3.0,3.0,6.0,5.0,5.

In [95]:
cols_w_null = []

# function that gives list of columns that contain null values
null_cols(acc_clean, cols_w_null)

acc_clean[cols_w_null]

""
0
1
2
3
4
...
674044
674045
674046
674047


In [96]:
acc_clean[acc_clean['pct_tl_nvr_dlq'].isnull()]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months


In [97]:
num_cols = [col for col in acc_clean.columns if '2' in col]

acc_clean[num_cols]

,delinq_2yrs,collections_12_mths_ex_med,acc_open_past_24mths,chargeoff_within_12_mths,grouped_subgrade_12_345,grouped_subgrade_123_45
0,0.0,0.0,4.0,0.0,6,6
1,0.0,0.0,6.0,0.0,3,3
2,1.0,0.0,4.0,0.0,3,3
3,0.0,0.0,7.0,0.0,1,1
4,0.0,0.0,5.0,0.0,4,4
...,...,...,...,...,...,...
674044,0.0,0.0,12.0,0.0,4,4
674045,0.0,0.0,6.0,0.0,3,3
674046,0.0,0.0,5.0,0.0,5,5
674047,0.0,0.0,4.0,0.0,5,5


In [98]:
num_cols.remove('grouped_subgrade_12_345')
num_cols.remove('grouped_subgrade_123_45')

num_cols

['delinq_2yrs',
 'collections_12_mths_ex_med',
 'acc_open_past_24mths',
 'chargeoff_within_12_mths']

In [99]:
acc_clean = acc_clean.drop(num_cols, axis=1)

acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,10.0,2,55000.0,0,Fully Paid,debt_consolidation,5.91,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,w,4421.723917,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,20701.0,1506.0,37.2,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2520.0,4429.08,3100.36,3,6,6,14,148.0,677.0,-1.0,38500.0,-1.0,-1.0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,10.0,2,180000.0,0,Fully Paid,debt_consolidation,14.67,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,f,21393.800000,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,30030.0,0.0,102.9,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14000.0,22952.88,16067.02,2,3,3,7,306.0,682.0,-1.0,126000.0,-1.0,-1.0
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,10.0,2,85000.0,0,Fully Paid,major_purchase,17.61,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,w,21538.508977,3.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,17700.0,13674.0,5.7,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2.0,3.0,3.0,9.0,3.0,3.0,2.0,8.0,93.3,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14000.0,22725.36,15907.75,2,3,3,6,202.0,707.0,-1.0,59500.0,-1.0,-1.0
3,10000.0,10000.0,10000.0,36,6.49,306.45,1,2,6.0,0,85000.0,0,Fully Paid,credit_card,13.07,10000.0,106.0,14.0,1.0,10464.0,34.5,23.0,w,10998.971575,10000.0,1.0,1,-1.0,-1.0,-1,0.0,8341.0,27957.0,1.0,35.0,17493.0,57.0,2524.0,46.0,30300.0,2.0,0.0,1997.0,8182.0,50.1,0.0,164.0,129.0,1.0,1.0,1.0,4.0,10000.0,1.0,10000.0,6.0,9.0,7.0,10.0,3.0,13.0,19.0,9.0,14.0,95.7,28.6,1.0,0.0,61099.0,27957.0,16400.0,30799.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7000.0,11032.20,7722.54,1,1,1,2,164.0,687.0,-1.0,59500.0,-1.0,-1.0
4,8000.0,8000.0,8000.0,36,11.48,263.74,2,10,10.0,2,42000.0,0,Fully Paid,credit_card,34.80,75.0,10000.0,8.0,0.0,7034.0,39.1,18.0,w,8939.580503,75.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,199696.0,2.0,10.0,106748.0,72.0,4725.0,49.0,18000.0,0.0,0.0,28528.0,9966.0,41.4,0.0,155.0,253.0,15.0,10.0,1.0,50.0,10000.0,10.0,10000.0,3.0,3.0,3.0,6.0,5.0,5.0,11.0,3.0,8.0,94.4,33.3,0.0,0.0,256513.0,113782.0,17000.0,135513.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5600.0,9494.64,6646.25,2,4,4,10,253.0,702.0,-1.0,29400.0,

# Income : * Ratios

In [100]:
acc_clean[(acc_clean['application_type']==1) &\
              (acc_clean['annual_inc_joint']==acc_clean['annual_inc'])]\
[['annual_inc_joint', 'annual_inc']]

,annual_inc_joint,annual_inc
391402,-1.0,-1.0
391403,-1.0,-1.0
391412,-1.0,-1.0


In [101]:
acc_clean[acc_clean['application_type']==2]

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months
237,15000.0,15000.0,15000.0,36,14.85,518.88,3,15,-1.0,2,37534.44,1,Fully Paid,debt_consolidation,31.44,10000.0,10000.0,13.0,0.0,39981.0,45.0,28.0,w,18710.562289,10000.0,1.0,2,61366.44,19.22,0,0.0,0.0,181626.0,0.0,29.0,0.0,-1.0,6579.0,45.0,88900.0,2.0,0.0,13971.0,31798.0,48.0,0.0,60.0,301.0,6.0,6.0,4.0,9.0,10000.0,6.0,10000.0,7.0,11.0,7.0,13.0,2.0,12.0,22.0,11.0,13.0,100.0,14.3,0.0,0.0,233900.0,39981.0,61200.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,10500.0,18679.68,13075.78,3,6,6,15,301.0,682.0,-1.0,37534.44,42956.508,-1.0
342,35000.0,35000.0,35000.0,60,17.97,888.20,4,19,10.0,2,150000.00,0,Fully Paid,debt_consolidation,25.11,20.0,10000.0,27.0,0.0,39913.0,85.3,45.0,w,43193.305750,25.0,1.0,2,192000.00,25.28,0,0.0,433.0,603735.0,5.0,11.0,139765.0,74.0,13130.0,78.0,46800.0,3.0,8.0,22361.0,3761.0,89.4,0.0,165.0,404.0,0.0,0.0,7.0,29.0,20.0,0.0,20.0,9.0,15.0,10.0,15.0,9.0,18.0,27.0,16.0,27.0,84.4,80.0,0.0,0.0,733890.0,180363.0,35600.0,166148.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,24500.0,53292.00,37304.40,4,8,8,19,404.0,672.0,-1.0,105000.00,134400.000,-1.0
874,25000.0,25000.0,25000.0,60,18.99,648.38,5,21,10.0,2,37856.00,1,Fully Paid,debt_consolidation,23.17,32.0,27.0,6.0,2.0,9624.0,89.1,13.0,w,34173.848630,32.0,1.0,2,74684.00,11.74,0,0.0,109.0,146727.0,1.0,17.0,20709.0,83.0,7631.0,85.0,10800.0,1.0,0.0,24455.0,176.0,98.2,0.0,153.0,231.0,25.0,17.0,3.0,72.0,10000.0,0.0,10000.0,2.0,2.0,3.0,4.0,3.0,4.0,7.0,2.0,6.0,84.6,66.7,0.0,0.0,211261.0,30333.0,9800.0,24961.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,17500.0,38902.80,27231.96,5,9,9,21,231.0,672.0,-1.0,37856.00,52278.800,-1.0
893,10500.0,10500.0,10500.0,36,9.80,337.83,2,8,0.0,2,65000.00,1,Fully Paid,debt_consolidation,21.49,10.0,10000.0,29.0,0.0,11156.0,83.9,43.0,w,12150.121933,65.0,1.0,2,92000.00,15.85,0,0.0,0.0,350643.0,24.0,1.0,167875.0,96.0,4970.0,95.0,13300.0,0.0,1.0,12091.0,2144.0,83.9,0.0,127.0,287.0,17.0,1.0,1.0,17.0,10.0,2.0,10.0,4.0,4.0,4.0,6.0,36.0,4.0,6.0,4.0,29.0,93.0,50.0,0.0,0.0,373179.0,179031.0,13300.0,184122.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,7350.0,12161.88,8513.32,2,4,3,8,287.0,667.0,-1.0,65000.00,64400.000,-1.0
960,26600.0,26600.0,26600.0,60,19.48,697.07,5,22,9.0,2,65000.00,1,Fully Paid,debt_consolidation,51.20,10000.0,10000.0,18.0,0.0,36896.0,94.1,43.0,w,34204.374736,10000.0,1.0,2,120000.00,28.45,0,0.0,0.0,154550.0,5.0,13.0,63469.0,46.0,6887.0,63.0,39200.0,0.0,4.0,8586.0,425.0,98.4,0.0,138.0,240.0,1.0,1.0,3.0,25.0,10000.0,1.0,10000.0,6.0,11.0,6.0,11.0,13.0,12.0,27.0,11.0,18.0,100.0,100.0,0.0,0.0,208930.0,1003

In [102]:
acc_clean[acc_clean['annual_inc_joint']>0]['annual_inc_joint']

237        61366.44
342       192000.00
874        74684.00
893        92000.00
960       120000.00
            ...    
673992    109000.00
674005    125000.00
674008     50000.00
674014    108000.00
674032     48828.00
Name: annual_inc_joint, Length: 12978, dtype: float64

In [103]:
acc_clean['annual_inc_joint'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
674044    False
674045    False
674046    False
674047    False
674048    False
Name: annual_inc_joint, Length: 674049, dtype: bool

## Giving weight to loan amount

In [104]:
# creating a new column
acc_clean['weighted_loan_amnt'] = acc_clean['loan_amnt']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_clean.loc[acc_clean['fico_score']<=560,
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 5

acc_clean.loc[((acc_clean['fico_score']>560) &\
               (acc_clean['fico_score']<=650)),
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 4

acc_clean.loc[((acc_clean['fico_score']>650) &\
               (acc_clean['fico_score']<=700)),
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 3

acc_clean.loc[((acc_clean['fico_score']>700) &\
               (acc_clean['fico_score']<=750)),
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 2

acc_clean.loc[acc_clean['fico_score']>750,
              'weighted_loan_amnt'] = acc_clean['weighted_loan_amnt'] * 1

acc_clean['weighted_loan_amnt']

0         10800.0
1         60000.0
2         40000.0
3         30000.0
4         16000.0
           ...   
674044    43200.0
674045     4000.0
674046    57600.0
674047    76000.0
674048    72000.0
Name: weighted_loan_amnt, Length: 674049, dtype: float64

## Giving weight to funded amount

In [105]:
# creating a new column
acc_clean['weighted_funded_amnt'] = acc_clean['funded_amnt']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_clean.loc[acc_clean['fico_score']<=560,
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 5

acc_clean.loc[((acc_clean['fico_score']>560) &\
               (acc_clean['fico_score']<=650)),
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 4

acc_clean.loc[((acc_clean['fico_score']>650) &\
               (acc_clean['fico_score']<=700)),
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 3

acc_clean.loc[((acc_clean['fico_score']>700) &\
               (acc_clean['fico_score']<=750)),
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 2

acc_clean.loc[acc_clean['fico_score']>750,
              'weighted_funded_amnt'] = acc_clean['weighted_funded_amnt'] * 1

acc_clean['weighted_funded_amnt']

0         10800.0
1         60000.0
2         40000.0
3         30000.0
4         16000.0
           ...   
674044    43200.0
674045     4000.0
674046    57600.0
674047    76000.0
674048    72000.0
Name: weighted_funded_amnt, Length: 674049, dtype: float64

## Giving weight to installment

In [106]:
# creating a new column
acc_clean['weighted_installment'] = acc_clean['installment']

# multiplying the new column by its perceived rank according to FICO score ranges
acc_clean.loc[acc_clean['fico_score']<=560,
              'weighted_installment'] = acc_clean['weighted_installment'] * 5

acc_clean.loc[((acc_clean['fico_score']>560) &\
               (acc_clean['fico_score']<=650)),
              'weighted_installment'] = acc_clean['weighted_installment'] * 4

acc_clean.loc[((acc_clean['fico_score']>650) &\
               (acc_clean['fico_score']<=700)),
              'weighted_installment'] = acc_clean['weighted_installment'] * 3

acc_clean.loc[((acc_clean['fico_score']>700) &\
               (acc_clean['fico_score']<=750)),
              'weighted_installment'] = acc_clean['weighted_installment'] * 2

acc_clean.loc[acc_clean['fico_score']>750,
              'weighted_installment'] = acc_clean['weighted_installment'] * 1

acc_clean['weighted_installment']

0          369.09
1         1912.74
2         1262.52
3          919.35
4          527.48
           ...   
674044     939.06
674045     127.20
674046    1304.40
674047    1748.36
674048    1693.68
Name: weighted_installment, Length: 674049, dtype: float64

### 1st borrower income

In [107]:
# Income to loan amount ratio

acc_clean['ratio_inc_loan'] = acc_clean['annual_inc'] / acc_clean['loan_amnt']

acc_clean['ratio_inc_weightedloan'] = acc_clean['annual_inc'] / acc_clean['weighted_loan_amnt']


# income reduced to loan amount ratio

acc_clean['ratio_redinc_loan'] = acc_clean['annual_inc_reduced'] / acc_clean['loan_amnt']

acc_clean['ratio_redinc_weightedloan'] = acc_clean['annual_inc_reduced'] /\
acc_clean['weighted_loan_amnt']


# Income to funded amount ratio

acc_clean['ratio_inc_funded'] = acc_clean['annual_inc'] / acc_clean['funded_amnt']

acc_clean['ratio_inc_weightedfunded'] = acc_clean['annual_inc'] / acc_clean['weighted_funded_amnt']


# income reduced to funded amount ratio

acc_clean['ratio_redinc_funded'] = acc_clean['annual_inc_reduced'] /\
acc_clean['funded_amnt']

acc_clean['ratio_redinc_weightedfunded'] = acc_clean['annual_inc_reduced'] /\
acc_clean['weighted_funded_amnt']


# installment to income ratio

acc_clean['ratio_install_inc'] = acc_clean['annual_inc'] / acc_clean['installment']

acc_clean['ratio_weightedinstall_inc'] = acc_clean['annual_inc'] / acc_clean['weighted_installment']


# installment to reduced income ratio

acc_clean['ratio_install_redinc'] = acc_clean['annual_inc_reduced'] /\
acc_clean['installment']

acc_clean['ratio_weightedinstall_redinc'] = acc_clean['annual_inc_reduced'] /\
acc_clean['weighted_installment']

### joint borrower income

In [108]:
# Income to loan amount ratio

acc_clean['ratio_incjoint_loan'] = acc_clean['annual_inc_joint'] / acc_clean['loan_amnt']

acc_clean['ratio_incjoint_weightedloan'] = acc_clean['annual_inc_joint'] /\
acc_clean['weighted_loan_amnt']


# income reduced to loan amount ratio

acc_clean['ratio_redincjoint_loan'] = acc_clean['annual_inc_joint_reduced'] /\
acc_clean['loan_amnt']

acc_clean['ratio_redincjoint_weightedloan'] = acc_clean['annual_inc_joint_reduced'] /\
acc_clean['weighted_loan_amnt']


# Income to funded amount ratio

acc_clean['ratio_incjoint_funded'] = acc_clean['annual_inc_joint'] /\
acc_clean['funded_amnt']

acc_clean['ratio_incjoint_weightedfunded'] = acc_clean['annual_inc_joint'] /\
acc_clean['weighted_funded_amnt']


# income reduced to funded amount ratio

acc_clean['ratio_redincjoint_funded'] = acc_clean['annual_inc_joint_reduced'] /\
acc_clean['funded_amnt']

acc_clean['ratio_redincjoint_weightedfunded'] = acc_clean['annual_inc_joint_reduced'] /\
acc_clean['weighted_funded_amnt']


# Income to installment ratio

acc_clean['ratio_incjoint_installment'] = acc_clean['annual_inc_joint']/\
acc_clean['installment']

acc_clean['ratio_incjoint_weightedinstallment'] = acc_clean['annual_inc_joint']/\
acc_clean['weighted_installment']


# Reduced income to installment ratio

acc_clean['ratio_redincjoint_installment'] = acc_clean['annual_inc_joint_reduced']/\
acc_clean['installment']

acc_clean['ratio_redincjoint_weightedinstallment'] = acc_clean['annual_inc_joint_reduced']/\
acc_clean['weighted_installment']

# Loan Amount vs Funded Amount

In [109]:
acc_clean['loan_minus_funded'] = acc_clean['loan_amnt'] - acc_clean['funded_amnt']

acc_clean['ratio_loan_funded'] = acc_clean['loan_amnt'] / acc_clean['funded_amnt']

# Dummification

In [110]:
# getting a list of cols w dtype object

cols_w_obj = list(acc_clean.select_dtypes(include=['object']).columns)

cols_w_obj

['loan_status', 'purpose', 'initial_list_status']

In [111]:
acc_clean.loc[acc_clean['initial_list_status']=='w', 'initial_list_status'] = 0
acc_clean.loc[acc_clean['initial_list_status']=='f', 'initial_list_status'] = 1

acc_clean[['initial_list_status']] = acc_clean[['initial_list_status']].astype(int)
acc_clean[['initial_list_status']]

,initial_list_status
0,0
1,1
2,0
3,0
4,0
...,...
674044,1
674045,1
674046,1
674047,1


In [112]:
acc_clean['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Default',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [113]:
acc_clean.loc[acc_clean['loan_status']=='Fully Paid', 'loan_status'] = 5
acc_clean.loc[acc_clean['loan_status']=='Charged Off', 'loan_status'] = 0
acc_clean.loc[acc_clean['loan_status']=='Current', 'loan_status'] = 4
acc_clean.loc[acc_clean['loan_status']=='Late (31-120 days)', 'loan_status'] = 3
acc_clean.loc[acc_clean['loan_status']=='In Grace Period', 'loan_status'] = 1
acc_clean.loc[acc_clean['loan_status']=='Late (16-30 days)', 'loan_status'] = 2
acc_clean.loc[acc_clean['loan_status']=='Default', 'loan_status'] = 0
acc_clean.loc[acc_clean['loan_status']=='Does not meet the credit policy. Status:Fully Paid',\
              'loan_status'] = 5
acc_clean.loc[acc_clean['loan_status']=='Does not meet the credit policy. Status:Charged Off',\
              'loan_status'] = 0

acc_clean['loan_status'] = acc_clean['loan_status'].astype(int)

acc_clean[['loan_status']]

,loan_status
0,5
1,5
2,5
3,5
4,5
...,...
674044,0
674045,0
674046,0
674047,0


In [114]:
acc_clean = acc_clean.drop('loan_status', axis=1)

acc_clean

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,10.0,2,55000.0,0,debt_consolidation,5.91,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,0,4421.723917,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,20701.0,1506.0,37.2,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2520.0,4429.08,3100.36,3,6,6,14,148.0,677.0,-1.0,38500.0,-1.0,-1.0,10800.0,10800.0,369.09,15.277778,5.092593,10.694444,3.564815,15.277778,5.092593,10.694444,3.564815,447.045436,149.015145,312.931805,104.310602,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.008128,-0.002709,-0.008128,-0.002709,0.0,1.0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,10.0,2,180000.0,0,debt_consolidation,14.67,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,1,21393.800000,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,30030.0,0.0,102.9,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14000.0,22952.88,16067.02,2,3,3,7,306.0,682.0,-1.0,126000.0,-1.0,-1.0,60000.0,60000.0,1912.74,9.000000,3.000000,6.300000,2.100000,9.000000,3.000000,6.300000,2.100000,282.317513,94.105838,197.622259,65.874086,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.001568,-0.000523,-0.001568,-0.000523,0.0,1.0
2,20000.0,20000.0,20000.0,36,8.49,631.26,2,6,10.0,2,85000.0,0,major_purchase,17.61,3.0,10000.0,8.0,0.0,826.0,5.7,15.0,0,21538.508977,3.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,141601.0,3.0,13.0,27111.0,75.0,640.0,55.0,14500.0,1.0,0.0,17700.0,13674.0,5.7,0.0,149.0,55.0,32.0,13.0,3.0,32.0,10000.0,8.0,10000.0,2.0,2

In [115]:
# dummification

acc_dum = pd.get_dummies(acc_clean, drop_first=True)

acc_dum

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_pymnt,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,3600.0,3600.0,3600.0,36,13.99,123.03,3,14,10.0,2,55000.0,0,5.91,30.0,10000.0,7.0,0.0,2765.0,29.7,13.0,0,4421.723917,30.0,1.0,1,-1.0,-1.0,-1,0.0,722.0,144904.0,2.0,21.0,4981.0,36.0,722.0,34.0,9300.0,3.0,1.0,20701.0,1506.0,37.2,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2520.0,4429.08,3100.36,3,6,6,14,148.0,677.0,-1.0,38500.0,-1.0,-1.0,10800.0,10800.0,369.09,15.277778,5.092593,10.694444,3.564815,15.277778,5.092593,10.694444,3.564815,447.045436,149.015145,312.931805,104.310602,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.000278,-0.000093,-0.008128,-0.002709,-0.008128,-0.002709,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,20000.0,20000.0,20000.0,36,9.17,637.58,2,7,10.0,2,180000.0,0,14.67,49.0,10000.0,12.0,0.0,87329.0,84.5,27.0,1,21393.800000,10000.0,1.0,1,-1.0,-1.0,-1,0.0,0.0,360358.0,2.0,18.0,29433.0,63.0,13048.0,74.0,94200.0,1.0,0.0,30030.0,0.0,102.9,0.0,142.0,306.0,10.0,10.0,4.0,12.0,10000.0,10.0,10000.0,4.0,6.0,4.0,5.0,7.0,9.0,16.0,6.0,12.0,96.3,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,14000.0,22952.88,16067.02,2,3,3,7,306.0,682.0,-1.0,126000.0,-1.0,-1.0,60000.0,60000.0,1912.74,9.000000,3.000000,6.300000,2.100000,9.000000,3.000000,6.300000,2.100000,282.317513,94.105838,197.622259,65.874086,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.000050,-0.000017,-0.001568,-0.000523,-0.001568,-0.000523,0.0,1.0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,20000.0,20000.0,20000.0,36,8.49,631.26

# Scaling the dataset

In [116]:
# we are preserving total_pymnt to use as our 

acc_totalpayment = copy.deepcopy(acc_dum[['total_pymnt']])

acc_dum = acc_dum.drop('total_pymnt', axis=1)

In [117]:
# creating copies

import copy

acc_full = copy.deepcopy(acc_dum)

In [118]:
# full scaling

from sklearn.preprocessing import MinMaxScaler

acc_dum_cols = list(acc_dum.columns)

for col in acc_dum_cols:
    acc_full[col] = MinMaxScaler().fit_transform(np.array(acc_full[col]).reshape(-1,1))
    
acc_full

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,0.078481,0.078481,0.09,0.0,0.338006,0.068867,0.333333,0.382353,1.000000,0.666667,0.005000,0.0,0.00691,0.0030,1.0000,0.094118,0.011494,0.000952,0.034367,0.080460,0.0,0.0030,0.0,0.0,0.0,0.0,0.0,0.2,7.899441e-05,0.028016,0.051724,0.0021,0.002912,0.098143,0.000931,0.189189,0.00093,0.121212,0.038462,0.021608,0.004116,0.112155,0.000004,0.149,0.012603,0.010724,0.012698,0.038462,0.0004,0.0069,0.0004,0.0069,0.083333,0.081967,0.054545,0.084507,0.02500,0.068493,0.077519,0.111111,0.094118,0.771287,0.009901,0.083333,0.011628,0.017805,0.002273,0.002172,0.006535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078481,0.052441,0.052441,0.50,0.384615,0.384615,0.541667,0.149,0.276008,0.0,0.003500,0.0,0.0,0.085427,0.085427,0.068867,0.002619,0.002619,0.001833,0.001833,0.002619,0.002619,0.001833,0.001833,0.002575,0.002575,0.001803,0.001803,0.000008,0.000007,0.000010,0.000010,0.000008,0.000007,0.000010,0.000010,0.000027,0.000009,0.000037,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.493671,0.493671,0.50,0.0,0.150312,0.368914,0.166667,0.176471,1.000000,0.666667,0.016365,0.0,0.01567,0.0049,1.0000,0.152941,0.011494,0.030063,0.095713,0.160920,1.0,1.0000,0.0,0.0,0.0,0.0,0.0,0.2,1.092592e-07,0.069672,0.051724,0.0018,0.017203,0.169761,0.016797,0.405405,0.00942,0.060606,0.019231,0.031345,0.000003,0.305050,0.000004,0.143,0.030406,0.029491,0.034921,0.096154,0.0012,1.0000,0.0010,1.0000,0.138889,0.114754,0.090909,0.084507,0.05000,0.136986,0.131783,0.155556,0.152941,0.963366,1.000000,0.083333,0.011628,0.038885,0.034260,0.028495,0.022100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.493671,0.287459,0.287459

In [119]:
col_names = list(acc_full.columns)
col_names.append('total_pymnt')

acc_full = pd.DataFrame(np.concatenate((acc_full, acc_totalpayment), axis=1),
                             columns=col_names)
acc_full

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,principal_70,total_expected,total_expected_70,grouped_grade_NO_EG,grouped_subgrade_12_345,grouped_subgrade_123_45,grouped_subgrade,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,total_pymnt
0,0.078481,0.078481,0.09,0.0,0.338006,0.068867,0.333333,0.382353,1.000000,0.666667,0.005000,0.0,0.00691,0.0030,1.0000,0.094118,0.011494,0.000952,0.034367,0.080460,0.0,0.0030,0.0,0.0,0.0,0.0,0.0,0.2,7.899441e-05,0.028016,0.051724,0.0021,0.002912,0.098143,0.000931,0.189189,0.00093,0.121212,0.038462,0.021608,0.004116,0.112155,0.000004,0.149,0.012603,0.010724,0.012698,0.038462,0.0004,0.0069,0.0004,0.0069,0.083333,0.081967,0.054545,0.084507,0.02500,0.068493,0.077519,0.111111,0.094118,0.771287,0.009901,0.083333,0.011628,0.017805,0.002273,0.002172,0.006535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078481,0.052441,0.052441,0.50,0.384615,0.384615,0.541667,0.149,0.276008,0.0,0.003500,0.0,0.0,0.085427,0.085427,0.068867,0.002619,0.002619,0.001833,0.001833,0.002619,0.002619,0.001833,0.001833,0.002575,0.002575,0.001803,0.001803,0.000008,0.000007,0.000010,0.000010,0.000008,0.000007,0.000010,0.000010,0.000027,0.000009,0.000037,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4421.723917
1,0.493671,0.493671,0.50,0.0,0.150312,0.368914,0.166667,0.176471,1.000000,0.666667,0.016365,0.0,0.01567,0.0049,1.0000,0.152941,0.011494,0.030063,0.095713,0.160920,1.0,1.0000,0.0,0.0,0.0,0.0,0.0,0.2,1.092592e-07,0.069672,0.051724,0.0018,0.017203,0.169761,0.016797,0.405405,0.00942,0.060606,0.019231,0.031345,0.000003,0.305050,0.000004,0.143,0.030406,0.029491,0.034921,0.096154,0.0012,1.0000,0.0010,1.0000,0.138889,0.114754,0.090909,0.084507,0.05000,0.136986,0.131783,0.155556,0.152941,0.963366,1.000000,0.083333,0.011628,0.038885,0.034260,0.028495,0.022100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

## Downsampling the dataset to reduce number of rows to reduce taxation on cpu

In [ ]:
target_3_train_downsample[['loan_status_binary']]

In [ ]:
from sklearn.utils import resample

label = target_3_train_downsample['loan_status_binary']

target_mean = label.mean()

target_fullypaid = label == 0

subsample_ind = resample(label[target_fullypaid].index.values, \
                         replace=False,
                         n_samples=int(len(target_3_train_downsample.index)*target_mean))

combined = np.concatenate([subsample_ind, label[~target_fullypaid].index.values])

acc_subsample = acc_full.iloc[combined]
target_subsample = target_3_train_downsample[['loan_status_binary']].iloc[combined]

acc_subsample = acc_subsample.reset_index()
acc_subsample = acc_subsample.drop('index', axis=1)

target_subsample = target_subsample.reset_index()
target_subsample = target_subsample.drop('index', axis=1)

target_subsample.mean()

In [130]:
downsampled.to_csv("idea_3_downsampled.csv.gz",
                index=False,
                compression="gzip")

target_downsampled.to_csv("idea_3_target_downsampled.csv.gz",
                        index=False,
                        compression="gzip")

downsampled_pred.to_csv("idea_3_downsampled_pred.csv.gz",
                index=False,
                compression="gzip")

target_downsampled_pred.to_csv("idea_3_target_downsampled_pred.csv.gz",
                        index=False,
                        compression="gzip")

In [112]:
grade_cols = [col for col in acc_subsample.columns if 'grade' in col]

grade_cols

['grade',
 'sub_grade',
 'grouped_grade_NO_EG',
 'grouped_subgrade_12_345',
 'grouped_subgrade_123_45',
 'grouped_subgrade']

In [113]:
acc_subsample_nogrades = acc_subsample.drop(grade_cols, axis=1)

acc_subsample_nogrades

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,dti,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_act_il,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_mths_since_last_major_derog,duration_of_credit_months,fico_score,sec_app_fico_score,annual_inc_reduced,annual_inc_joint_reduced,sec_app_duration_of_credit_months,weighted_loan_amnt,weighted_funded_amnt,weighted_installment,ratio_inc_loan,ratio_inc_weightedloan,ratio_redinc_loan,ratio_redinc_weightedloan,ratio_inc_funded,ratio_inc_weightedfunded,ratio_redinc_funded,ratio_redinc_weightedfunded,ratio_install_inc,ratio_weightedinstall_inc,ratio_install_redinc,ratio_weightedinstall_redinc,ratio_incjoint_loan,ratio_incjoint_weightedloan,ratio_redincjoint_loan,ratio_redincjoint_weightedloan,ratio_incjoint_funded,ratio_incjoint_weightedfunded,ratio_redincjoint_funded,ratio_redincjoint_weightedfunded,ratio_incjoint_installment,ratio_incjoint_weightedinstallment,ratio_redincjoint_installment,ratio_redincjoint_weightedinstallment,loan_minus_funded,ratio_loan_funded,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,initial_list_status_w,total_pymnt
0,0.367089,0.367089,0.374375,0.0,0.485202,0.312740,1.000000,1.000000,0.009546,0.0,0.00838,0.0022,1.0000,0.087912,0.016129,0.006397,0.255670,0.103448,0.0022,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.000001,0.002569,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.00338,0.000000,0.0000,0.003671,0.001902,0.350507,0.000004,0.255172,0.010602,0.020501,0.032967,0.038462,0.0008,1.0000,0.0004,0.0022,0.083333,0.078125,0.046875,0.070423,0.03125,0.071429,0.085271,0.108696,0.087912,1.000000,1.000000,0.083333,0.016129,0.004188,0.007644,0.012501,0.003807,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.213365,0.382166,0.000000,0.006682,0.000000,0.000000,0.244966,0.244966,0.207802,0.001200,0.001800,0.000840,0.001260,0.001200,0.001800,0.000840,0.001260,0.001119,0.001678,0.000783,0.001175,0.000009,0.000006,0.000013,0.000008,0.000009,0.000006,0.000013,0.000008,0.000028,0.000009,0.000040,0.000013,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16643.221785
1,0.240506,0.240506,0.250000,0.0,0.182243,0.185497,1.000000,0.666667,0.007273,1.0,0.00658,0.0052,0.0045,0.054945,0.032258,0.001814,0.197423,0.097701,0.0075,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.004856,0.018869,0.000000,1.0000,0.000000,0.000000,0.000000,0.000000,0.00125,0.000000,0.0000,0.047174,0.014004,0.149493,0.000004,0.175172,0.015303,0.059226,0.095238,0.115385,0.0025,0.0075,0.0022,0.0075,0.083333,0.046875,0.046875,0.084507,0.02500,0.035714,0.069767,0.065217,0.054945,0.868317,0.504950,0.166667,0.016129,0.018550,0.004217,0.016449,0.005375,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.182884,0.212314,0.000000,0.007273,0.000000,0.000000,0.244966,0.244966,0.185736,0.001371,0.001372,0.001371,0.001372,0.001371

In [114]:
acc_subsample_nogrades.to_csv("acc_2_train_downsample_nogrades.csv.gz",
                              index=False,
                              compression="gzip")

# Viviann and Ali's Reduced Dataset

In [115]:
# Random Forest

acc_VC_rf = copy.deepcopy(acc_subsample[['int_rate', 'dti', 'avg_cur_bal', 'revol_bal',
                                         'duration_of_credit_months', 'ratio_inc_funded',
                                         'ratio_redinc_loan', 'installment', 'annual_inc',
                                         'loan_amnt', 'open_acc', 'grouped_subgrade_12_345',
                                         'emp_length', ]])

acc_VC_gb = copy.deepcopy(acc_subsample[['grouped_subgrade_12_345', 'grade', 'term',
                                         'ratio_inc_funded', 'max_bal_bc', 'int_rate', 'fico_score',
                                         'dti', 'emp_length', 'home_ownership', 'revol_bal',
                                         'annual_inc', 'duration_of_credit_months']])

acc_VC_rf.to_csv("acc_VC_rf.csv.gz",
                 index=False,
                 compression="gzip")

acc_VC_gb.to_csv("acc_VC_gb.csv.gz",
                 index=False,
                 compression="gzip")

# Theodore's Reduced Dataset

In [121]:
acc_TC = copy.deepcopy(acc_subsample[['annual_inc', 'annual_inc_joint', 'application_type', 'dti',
                                      'dti_joint', 'emp_length', 'duration_of_credit_months',
                                      'funded_amnt', 'fico_score', 'home_ownership', 'installment',
                                      'int_rate', 'loan_amnt', 'term', 'verification_status',
                                      'verification_status_joint', 'sec_app_fico_score',
                                      'sec_app_duration_of_credit_months', 'grade', 'sub_grade']])


acc_TC.to_csv("acc_TC.csv.gz",
              index=False,
              compression="gzip")

# Very Reduced Dataset

In [117]:
cols_w_purpose = [col for col in acc_subsample.columns if 'purpose' in col]
acc_subsample[cols_w_purpose]

,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269355,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269356,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269357,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269358,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
cols_w_purpose = cols_w_purpose + ['annual_inc', 'fico_score', 'term', 'loan_amnt', 'home_ownership',
                                   'int_rate', 'all_util']

cols_w_purpose

['purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'annual_inc',
 'fico_score',
 'term',
 'loan_amnt',
 'home_ownership',
 'int_rate',
 'all_util']

In [123]:
acc_vred = copy.deepcopy(acc_subsample[['annual_inc', 'fico_score', 'term', 'loan_amnt',
                                        'home_ownership', 'int_rate', 'all_util','dti',
                                        'total_pymnt']])


acc_vred.to_csv("acc_vred.csv.gz",
              index=False,
              compression="gzip")